In [29]:
from data import *

In [2]:
def mod_mul_standard(a, b, p):
    result = (a * b) % p
    if result > (p>>1):
        result -= p
    return result

def mod_add(a, b, p):
    result = (a + b)
    if result > (p>>1):
        result -= p
    return result

def mod_sub(a, b, p):
    result = (a - b) + p
    if result > (p>>1):
        result -= p
    return result


def check_arrays(a,b,p):
    if a == b:
        print("exactly equal")
        return True
    print("not exactly equal")

    if len(a) != len(b):
        print("lengths not equal")
        return False


    for i in range(len(a)):
        if (a[i] - b[i]) % p != 0:
            print("element", i, "not equal even in modulus class")
            return False
    print("But equal in modulus class")
    return True

    



In [3]:
def Mu(b,q):
    mu = 2*int(((b << 31)/(2*q))+0.5)
    return mu


def Barrett_mul(a, b, p, mu_b):
    # Perform the multiplication
    z = (a * b) % (1 << 32)
    if z >= (1 << 31):
        z -= (1 << 32)  # Convert to signed 32-bit integer

    # Perform the multiplication for mu_b and get the high 32 bits
    temp = (a * (mu_b << 1)) % (1 << 64)
    t_high = (temp >> 32) % (1 << 32)
    if t_high >= (1 << 31):
        t_high -= (1 << 32)  # Convert to signed 32-bit integer



    
    result = z - t_high * p
    # Simulate 32-bit signed integer overflow for the result
    result %= (1 << 32)
    if result >= (1 << 31):
        result -= (1 << 32)  # Convert to signed 32-bit integer

    return result

In [4]:
def CT_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    # Second layer
    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    return t0, t1, t2, t3

def GS_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    # Second layer
    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    return t0, t1, t2, t3


In [5]:
print(f)

[1, 0, 3, 7, 2, 0, 0, 0, 3, 3, 1, 2, 0, 1, 0, 0, 0, 3, 0, 1, 1, 1, 0, 5, 3, 3, 1, 3, 4, 1, 0, 3, 4, 1, 1, 2, 1, 1, 1, 1, 4, 1, 2, 2, 1, 2, 1, 2, 2, 4, 7, 1, 3, 6, 0, 1, 0, 0, 0, 0, 6, 2, 7, 0, 4, 3, 0, 0, 1, 1, 0, 6, 2, 2, 1, 0, 0, 0, 6, 3, 1, 1, 0, 2, 4, 0, 0, 0, 3, 6, 1, 5, 2, 2, 7, 1, 3, 0, 2, 5, 1, 1, 5, 2, 1, 0, 1, 3, 0, 0, 6, 4, 4, 0, 2, 2, 0, 1, 3, 1, 4, 2, 2, 4, 3, 3, 0, 0, 2, 2, 2, 0, 0, 5, 3, 0, 0, 1, 1, 3, 1, 1, 1, 3, 0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 2, 0, 3, 3, 1, 4, 1, 3, 0, 1, 3, 2, 1, 2, 2, 1, 5, 0, 3, 7, 1, 1, 1, 3, 3, 0, 1, 0, 3, 1, 1, 0, 1, 1, 2, 3, 4, 0, 4, 0, 1, 3, 0, 1, 2, 0, 1, 5, 1, 1, 0, 2, 4, 0, 3, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 1, 3, 3, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 1, 4, 0, 0, 0, 0, 2, 0, 0, 6, 0, 3, 0, 1, 0, 3, 3, 1, 1, 2, 2, 3, 0, 2, 2, 4, 0, 0, 2, 0, 1, 1, 1, 4, 4, 2, 0, 2, 2, 1, 3, 5, 3, 2, 1, 0, 4, 0, 0, 4, 1, 1, 5, 1, 4, 0, 4, 4, 4, 2, 3, 0, 0, 1, 4, 1, 0, 0, 1, 3, 2, 1, 1, 2, 3, 5, 2, 2, 0, 1, 1, 7, 8, 3, 2, 2, 2, 3, 1, 0, 0, 1, 1, 0, 3, 0, 3, 0, 

In [6]:
# line 169 
logn_top = 10
n = 1 << logn_top

hhn = n >> 2
power_index = 1
for i in range(0, hhn):
    f[i], f[i+hhn], f[i+2*hhn], f[i+3*hhn] = CT_4pts(P[0], f[i], f[i+hhn], f[i+2*hhn], f[i+3*hhn],  omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    g[i], g[i+hhn], g[i+2*hhn], g[i+3*hhn] = CT_4pts(P[0], g[i], g[i+hhn], g[i+2*hhn], g[i+3*hhn],  omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])

In [7]:
correct = [-4755557, 0, 15052008, -3860830, -4548301, -6283050, 0, -6419528, 2203955, -16501632, 8010713, 207257, 829020, 3387712, 13950029, 10558397, 7248548, -3860834, -9800154, 4149876, -6412442, -11457038, -3860837, 14844755, 3, -14555709, 11109386, 550991, -6068706, -894720, 15395738, -8616392, 207259, -7721673, 10976829, 10089194, 11742157, 10640181, 14501018, -11038607, 550992, 9456422, 8561702, 10640182, -9249165, 1101978, -4211654, 15946728, 4149877, 414514, -16083197, 8561701, -7939932, 4149881, 16579497, 6697561, -9592899, -13117088, -6626783, -6626783, -3571793, 4020484, -9936625, -3309849, 5595588, 5595587, 6353827, -15399659, 1, 10007410, -8698178, 8010718, 1652966, 11534903, 11109386, 13606296, -6834038, 550988, 3173377, -2207870, 8905434, 1101977, 2278650, -13660989, 12078808, 13950029, 1942002, 15052005, 6146575, -2551600, 1309232, -15950642, 4700865, -9800152, 8561707, 10983914, 15395741, -5524807, -8698176, -3309844, -12351759, 2203953, -13109998, 2, 1727663, 10983913, 15395739, -12008024, 12636877, -3102594, -15739465, -15188479, -7721670, 15052005, 7115993, -9249164, -894721, -14086506, -7596199, -12902747, 8905437, 8905435, 840028, 16497718, 1652967, -894718, 14501017, 16497714, 12636879, 2411209, 15602995, 16028509, -8490923, 5595589, 1101979, 13055308, -13328264, 9112689, -15739470, -5517719, -15188482, -8698177, 1183760, -15950644, 2071395, -13879251, 10007409, -12559015, -2684163, 9663678, 8905435, -3653582, 16021426, -13117086, -10913136, 5044596, 13950030, 10983915, 0, -13117085, 11534904, -4755557, -9592895, -15188482, -10694872, -3653582, 550989, -5986924, -894719, -894720, 14157286, 10007411, 550989, -7596197, 7112070, -10480532, -6283043, -10906050, -8698177, 9456422, -15399656, -5181071, 2071395, 207256, 2622383, 7123079, -7939934, 16579498, 1727662, -9249165, -473125, -2551604, -8490920, -894717, -9249166, 11534905, 13950029, -14430239, -5099288, 4489687, 1309232, -13660989, 1101976, 2203953, -6075790, -6834037, 2622384, 1309231, 2, 1391018, -3653582, -343730, 0, -5524805, -15188483, -2758860, 1309231, -2758861, -3860837, -9800152, 2285735, 9112688, -15188482, 10007412, 3305931, 11109387, -343732, 1, 15946726, 4700863, -13660991, 3305930, 15395740, 6146572, 12078805, -14637494, -7170682, -2758861, -9249166, 15052005, -894721, 1101978, -13660991, 8561700, 6, 10640180, 3, -9800154, 10558398, 15395738, 8561703, 5044599, -3309848, 10640181, -9800152, -9800152, 4755559, 0, -12848050, 3860844, 6752257, 7385026, 0, 10827432, 2203955, -12640795, -8010711, 1996699, 7986788, 1020194, -13950029, -7252469, -2840644, 3860840, 9800154, -4149874, 10820348, 15864944, 3860837, -14844745, 3, 14555715, -6701480, 550991, 9374642, 894722, -14293762, 8616398, 1996701, 7721675, -10976827, -10089190, -8436227, -9538203, -13399040, 12140585, 550992, -8354444, -7459722, -9538202, 10351143, 1101978, 5313632, -13742772, -4149873, 3993398, -16365150, -7459723, 13449818, -4149869, -15477521, -3391631, 11796851, 14219064, 8830735, 8830735, 3571805, -714552, 13242567, 4411825, -4493604, -4493605, -1945923, -11538822, 1, -7803456, 10902130, -8010706, 1652966, -10432923, -6701480, -12504320, 6834038, 550988, -969413, 5513804, -8905432, 1101977, -74698, 13660993, -9874848, -13950029, 1363926, -12848053, -3942617, 6959516, 3098674, -12089805, -3598885, 9800156, -7459717, -10983912, -14293759, 9932711, 10902132, 4411830, 16759665, 2203953, 14211984, 2, -625685, -10983913, -14293761, 15313958, -9330949, 6408522, 15739477, 16290463, 7721678, -12848053, -7115989, 10351144, 894721, -16157901, 12004109, 16208677, -8905429, -8905431, 261952, -13191782, 1652967, 894724, -13399041, -13191786, -9330947, 4200651, -12297063, -16028509, 12898827, -4493603, 1101979, -13055308, -13610217, -6908735, 15739472, 9925629, 16290460, 10902131, -1183758, -12089807, -2071395, -14161204, -7803457, 14762967, 2684163, -6357746, -8905431, 5857534, -16021422, 14219066, 16423016, -5044596, -13950028, -10983911, 0, 14219067, -10432922, 4755559, 11796855, 16290460, 10694878, 5857534, 550989, 7088906, 894723, 894722, -11953330, -7803455, 550989, 12004111, 601762, 12684490, 7385033, 16415932, 10902131, -8354444, -11538819, 8487005, -2071395, 1996698, -1520407, -7123073, 13449816, -15477520, -625686, 10351143, 4881031, 6959512, 12898830, 894725, 10351142, -10432921, -13950029, -14712192, 6201270, 2122169, 3098674, 13660993, 1101976, 2203953, 9381728, 6834039, -1520406, 3098673, 2, 812942, 5857534, 1445712, 0, 9932713, 16290459, 4962814, 3098673, 4962813, 3860837, 9800156, -81783, -6908736, 16290460, -7803454, 3305931, -6701479, 1445710, 1, -13742774, -3598887, 13660991, 3305930, -14293760, -3942620, -9874851, -16708889, 8272666, 4962813, 10351142, -12848053, 894721, 1101978, 13660991, -7459724, 6, -9538204, 3, 9800154, -7252468, -14293762, -7459721, -5044593, 4411826, -9538203, 9800156, 9800156, -9189276, 0, 9386740, -5805636, -13674885, -9659791, 0, 15470314, -2203949, 11668398, 4763545, -4485608, 15607897, -5668059, 10488713, 6494214, 137583, -5805640, -11530812, -1042098, 16351466, 14009931, -5805643, 13872352, 3, 12830251, 5943227, -550985, -13264246, -3383633, 13321359, -14994917, -4485606, -11611285, 7185554, 4683072, 3030107, 4132083, 9937726, 14701270, -550984, 7596191, 4212558, 4132084, -12081799, -1101974, -10621415, 12770373, -1042097, -8971216, 12359741, 4212557, 15880954, -1042093, 9857251, 688572, -16016422, 14781743, -13594413, -13594413, -12653379, -8581178, 13599300, -6356631, 1790551, 1790550, -3246051, 10566419, 1, 7045203, -12632788, 4763550, -1652962, 7515718, 5943227, 6554091, -9108803, -550988, -2063595, -7458604, 8147179, -1101975, -5447235, 16213884, 6083581, 10488713, -8500706, 9386737, 1239562, -11393223, -5587585, 11117412, -1593085, -11530810, 4212563, 8066705, 13321362, -14696389, -12632786, -6356626, 10626297, -2203951, 15662899, 2, -4896246, 8066704, 13321360, 14560921, 6413740, -10842241, 16294362, 15743372, -11611282, 9386737, 1379912, -12081798, -3383634, 14641393, -13734761, 11177285, 8147182, 8147180, -7398728, 12219387, -1652961, -3383631, 9937725, 12219383, 6413742, -6689560, 8835751, 10408239, 16431939, 1790552, -1101973, 7105079, 9604794, 3661569, 16294357, -13815235, 15743369, -12632787, -3464107, 11117410, -961625, 10155783, 7045202, 15111906, -10150902, 3110582, 8147180, -10291253, 9527090, 14781745, 12577791, 2341535, 10488714, 8066706, 0, 14781746, 7515719, -9189276, -16016418, 15743369, -14914443, -10291253, -550987, -15626379, -3383632, -3383633, 6003105, 7045204, -550987, -13734759, -8282649, 15031435, -9659784, 13458943, -12632787, 7596191, 10566422, -10761764, -961625, -4485609, -1512613, 2261064, 15880952, 9857252, -4896247, -12081799, -11473702, -11393227, 16431942, -3383630, -12081800, 7515720, 10488713, 10706771, -13123896, -6770036, -5587585, 16213884, -1101976, -2203951, -14145396, -9108802, -1512612, -5587586, 2, -7949714, -10291253, -3934619, 0, -14696387, 15743368, -6907618, -5587586, -6907619, -5805643, -11530810, -4566084, 3661568, 15743369, 7045205, -3305925, 5943228, -3934621, 1, 12770371, -1593087, 16213882, -3305926, 13321361, 1239559, 6083578, 15192381, -12162270, -6907619, -12081800, 9386737, -3383634, -1101974, 16213882, 4212556, 6, 4132082, 3, -11530812, 6494215, 13321359, 4212559, 2341538, -6356630, 4132083, -11530810, -11530810, 9189278, 0, -11590686, 5805650, 11470937, 8557815, 0, 13672119, -2203949, -16076296, -4763543, 2281660, 9126632, 1260157, -10488713, -9800142, -4545487, 5805646, 11530812, 1042100, 12790969, 15132504, 5805643, -13872342, 3, -12830245, -10351129, -550985, 9958326, 3383635, -14423335, 14994923, 2281662, 11611287, -7185552, -4683068, -6336033, -5234057, -11039700, -15803244, -550984, -8698165, -5314530, -5234056, 10979825, -1101974, 9519441, -14974321, 1042101, 4563320, -13461703, -5314531, 12159509, 1042105, -10959227, -3994498, 13812470, -15883719, 11390461, 11390461, 12653391, 5275254, 16645123, 5254655, -2892519, -2892520, -1161853, 16372062, 1, -9249153, 10428836, -4763538, -1652962, -8617690, -10351129, -7656067, 9108803, -550988, -140345, 4152682, -8147177, -1101975, 3243283, -16213880, -8287525, -10488713, 5194778, -11590689, -3443508, 6985331, 1179683, -16627282, 491113, 11530814, -5314525, -8066703, -14423332, 10288485, 10428838, 5254660, -15034199, -2203951, -16764865, 2, 3794272, -8066704, -14423334, 15683494, -9719668, 7536313, -16294350, 16704997, 11611290, -11590689, -1379908, 10979826, 3383634, 15603018, 9326863, -14483211, -8147174, -8147176, 6296756, -15525307, -1652961, 3383637, -11039701, -15525311, -9719666, 77708, -12141675, -10408239, 12710494, -2892518, -1101973, -7105079, -16216650, -5865519, -16294355, 9407337, 16704994, 10428837, 3464109, -16627284, 961625, -15665661, -9249154, 16234479, 10150902, -6416506, -8147176, 8087301, -9527086, -15883717, 15462666, -2341535, -10488712, -8066702, 0, -15883716, -8617689, 9189278, 13812474, 16704994, 14914449, 8087301, -550987, 14524409, 3383636, 3383635, -8207053, -9249152, -550987, 9326865, 568817, 16314956, 8557822, 14581516, 10428837, -8698165, 16372065, 7455842, 961625, 2281659, 410637, -2261058, 12159507, -10959226, 3794271, 10979825, 7065800, 6985327, 12710497, 3383638, 10979824, -8617688, -10488713, -15114673, 12021926, 158180, 1179683, -16213880, -1101976, -2203951, 10839478, 9108804, 410638, 1179682, 2, 5745770, 8087301, 2832649, 0, 10288487, 16704993, 4703668, 1179682, 4703667, 5805643, 11530814, 2362132, -5865520, 16704994, -9249151, -3305925, -10351128, 2832647, 1, -14974323, 491111, -16213882, -3305926, -14423333, -3443511, -8287528, 16154006, 11060302, 4703667, 10979824, -11590689, 3383634, -1101974, -16213882, -5314532, 6, -5234058, 3, 11530812, -9800141, -14423335, -5314529, -2341532, 5254656, -5234057, 11530814, 11530814] 
check_arrays(f, correct, P[0])

exactly equal


True

In [8]:
# line 181
n = hhn
hhn = n >> 2
power_index = 4

for j in range(0, 4):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = CT_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = CT_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1



In [9]:
correct = [4682066, 10873204, 13273164, 10135471, 2001739, -150128, -3254723, -7673365, -12242520, -15739090, -1655549, -13355031, -6795581, -9852558, -11746565, -905274, 7683850, -3569559, 12388470, 4006604, 8229749, -58935, -14151395, 4673014, 4482536, -6852998, 2343844, 6673172, -7028062, -6252350, 4876281, 13965154, 5936833, -10681511, -8713870, 15673095, -8968259, -8066979, 10128624, -15936683, 4313878, 13099931, 10591110, -11505933, -8465235, -14331747, 7389454, 2999424, 2779421, -10711266, -5966324, -15453640, 11154479, 1937995, -2802119, -5548863, -6051527, -3041226, 481952, -3568106, -7145888, 6562253, 13717904, 11977358, -1183074, -7691370, -1239707, 8635940, 7555631, -12372847, 14625396, 4435501, 1655734, 5403629, -8392102, 7872512, -7983846, -4366661, -11234266, 2829334, -6512873, 2141073, 9105665, 1214228, -1341332, 6813423, -10551091, 10218462, 2326636, 12447819, -2281706, -2005430, -15893553, -11421814, -7635142, 877452, -14406278, -7903601, -14015453, -11932174, 4687558, 14549307, -7676164, 11468577, 9280954, -12613727, -3415329, 1518368, -3032334, 4280884, -9967651, -14531684, -4871005, 1175018, -13644376, -5691648, -3045130, -7793292, -10915343, -13769772, 16721557, 7126688, 11919215, -9741965, 14062339, 4402799, 2889608, 14156769, 3825805, 6989792, 2689082, 3496557, 9749598, -8242865, 5086107, -15148700, 8177000, -13023945, -13425910, -10263233, 12137363, -11895728, -9069000, 12606779, 4883973, 7514022, 5014584, -16031536, 7657793, -15321997, -15399645, -12635652, -4358495, -240493, -4738480, 16409005, -7736177, 4773744, -10315723, 15354521, 8800291, -15679464, -16045619, 16700473, 12485661, 3424057, 1421312, 11968224, -7144978, -15174985, 8784236, 14529844, -9052658, 4729349, 2929580, -14111416, 16551018, 3959772, -3518069, 14638897, 9877803, -4080753, 14653488, 8702911, -2034270, 13260504, -5264510, 11026689, -8312375, 12266106, 13956429, 15514619, 7203312, -10171626, 11935156, -4160951, -3949835, -4366360, -16456780, -7291548, 11225606, -9096785, -11584261, 16574780, 5958144, 6115458, -12801064, -5847588, -10611095, 12021465, 1391339, -6140129, -6645641, -3710306, -8891554, -9977478, -2450665, 3523510, 15563549, 14677554, 6382968, 9321540, 7556862, 2437979, 498190, 3377884, 15581584, -13634955, 5213331, -895998, -12970374, 15446128, -4245886, -14586205, -15263546, 4468112, -16446433, 9729426, 16352338, -11220423, 2140074, 7234532, -7653682, 7202858, -16196543, -7014763, -1718712, 3855631, -13457019, -2713644, -93452, 9326587, -12891248, -7149222, -3209767, 12212532, 4487280, 2711227, 16722498, -10047585, 6031232, 12052129, -403370, -16167578, 7485347, 16666960, 13137192, -14543115, 9618387, -6068949, 2484967, 11773202, -15264199, -14374935, 4782405, -9562985, 6220727, 2139561, -10095574, 12705467, 7375927, 13375615, -11505604, -4847219, -7103785, -11413281, -12804572, 14875087, -4967207, 515969, -6134546, -2825277, -3959819, 10745059, 9381335, -6686710, 4867373, -2774359, -1251235, 1479652, -12682953, 3068902, 14818445, 5800777, 7786297, 2704258, -11669627, 13089246, 6085175, -6344114, -3981969, -14659643, 14803803, -3644858, 108471, 7470410, -9088965, 15606513, 9735334, 13919056, -7520310, -3763932, -1552342, -7258677, -4615435, 11420580, -11090685, 12212254, -11874536, 6309713, -5030312, 5352393, -7975053, 9313193, -4882409, 8247861, 3040714, -4718382, 11753524, 12681204, 3313851, 16463941, 12267410, 9325382, 8122422, 5487608, 14073819, -10758, 3719040, 15383198, -15782952, 2733734, -16693230, -13562645, -9144820, 1866992, -7046165, -11651463, 3086947, 13836428, -8919189, 3837776, 5703990, -2371971, 5550927, -530913, 3337894, -15815684, 6530805, 12600116, 13693520, -15540632, -16668419, 1898219, 35067, 15832272, -2660505, -1374640, 11180307, -9120865, 11930244, -10923554, 9907594, -9043691, 10798794, 3902514, 3297650, 6913098, -6950009, -4303613, -9017607, 931517, -13777890, 5196406, 8187556, -9386928, 8099171, -4181726, 8417309, 2244726, 538028, -5264321, 10320890, 6753759, -1823028, 649907, -8683909, -10868330, 5904595, 6651517, -893722, 11028955, -13230018, 2679838, 14915981, 3550297, 14500059, 13379809, 9180870, -9067419, 15097134, -11904475, 15740958, 16462059, 14387335, -13568958, 4622174, -13954804, 13700199, 4089717, 4233732, 8110118, -12986006, -8981053, 4580455, 1257, 1213012, -12182958, 6957883, -2961506, -9704662, -8564480, 404237, -6100464, -4494158, -3301769, 4078424, -2669073, 11256472, -2849809, -2759332, -7713797, -1007097, 10371008, -13038675, 12783198, -6567438, -11635585, -14786943, -6985891, -11345690, 5018258, 5280691, 15825009, 322881, 6250661, 12343797, 7078863, 2019505, 10656030, -11594916, -3440825, 12147405, -842217, 5069924, -3967014, -12987343, -3070867, -9953168, 13306474, 15147182, -2996770, -4258475, 3219281, -4285721, 8208404, -14704726, -8599774, -8131551, 12638102, -5363480, 10244652, 1286062, -2219794, -1135543, 14034375, 15619749, 9650271, -4964274, 6161574, -3931170, 29377, 2179609, -11777647, 11198431, 3359560, -9920412, 9523522, 15940099, -4239398, -4301421, 15441008, 3226388, -8030295, -6419536, -5395288, -13030621, 1556987, -7653266, -11286938, 14931701, 13589811, 11323015, 41847, 12006719, -16383569, -4490190, 13436765, 1868353, -10588176, 2671048, -1290317, -3615053, 14018294, 6500085, 15363203, -5445619, 8261481, 9922779, 14211389, 11105438, 15666179, -3129706, 6923474, 1030901, -4440824, -7970825, 13808281, -10557205, 4464483, -4626174, 16770724, -697326, 4242979, -7197321, 1016958, -14367042, 3159315, 10186524, -5175427, -1690985, 4329158, -11805935, -7350119, -5262268, -11716348, -208743, 6126675, 15745583, 3193043, 13871231, 10705880, -4256876, -7207714, 336871, 15472240, 15675080, 11896989, 9669157, 7487612, -1926332, 11125546, 899209, 12729784, -10056275, 8195645, 9578642, -14561605, -9322692, -5892477, 12515061, 2371745, -16366348, 16602196, -200035, 10014137, 16208791, 8080210, -9445119, -5666526, -5783404, 9630409, -1650546, 13114858, -2525440, 364229, -3777913, -9145604, 8425543, 11418537, -11710014, 576433, -6708513, -14248344, -13418729, 11212731, 8047697, -16134574, 10934956, 3201468, -14166574, 12479068, -4982017, -5487126, -12314808, 5004751, -7964848, 16099509, -62076, 10604990, 5150084, -10995165, -1806911, 7787345, 513079, 4989209, -4148322, -14538573, 12219109, 12127109, 8776273, -14330715, 7225760, -7331458, -4576499, -16198880, -2526384, -15919112, -13560516, -1619793, -9479623, -1041391, -15800874, -15105808, 15543310, -2846507, -7326040, 15725548, -11003293, -8667247, 4298209, 15425330, 13751594, 2017341, 5358567, -3904794, -846730, -6835311, -2990359, -2357296, -14812872, 10796732, 6336622, 13634659, -14413617, 16612380, -190442, 14553563, 12990192, -4795746, -3264368, 11704560, -1299713, -9857014, -958642, 12757651, 8305272, -14063908, 4123825, 9037910, -14130445, -16351917, -1748128, -15446127, -13143967, 2393786, -8949011, 6136627, 13471275, 8939132, -138945, -2641311, 9174066, 11917371, 2665829, 10464021, -6478758, 15566927, -1615751, -1572480, 9280441, 9256848, -130706, -7042408, 7632146, -2381283, 8447761, 14709618, -15163666, 13014853, -11175970, -11411791, -4040631, -4224965, 14963580, -15034447, -478795, 10225053, 5870468, 3783571, 16347405, 10855461, 8454051, 6342693, 7905253, -1453596, -185153, 43162, -4264002, -12264806, -3973749, 11758630, 11255278, 2067034, -6289939, -2978538, 14412310, -11383413, 14715399, 746407, 11105235, 8707279, 5713580, 12843991, 16239518, -9097582, 3495614, 12389113, 14081293, 5228992, -1954675, 8459620, -15132563, -11911199, -8519036, -9937114, -883183, 2157430, -501367, -12304631, 1389649, -13428663, -2818271, 13046511, 14711185, 5552186, -15279766, 4137892, -7569384, -10113843, -1613572, 13712315, -10488024, -14879085, -9251541, 2778270, -8792687, -13691989, -8617915, -16689235, 6905529, 11355132, 6912490, -16034867, 16186752, -4962225, 1324580, -771539, 6613512, 11175858, 15467496, 14644160, -5211348, -15398930, -2266449, -13787945, -5739421, -8448535, 15006342, 13502499, -4749995, -10556471, 5813935, -13579171, -14485163, 9648967, 4303444, -5209706, 16594018, 12391598, -13769916, -6350053, 14223628, 5599786, -16170775, 5668406, -13330209, -9007631, -15593643, 7197715, 5967718, 366592, 1785464, 12674538, -3946478, -3777692, 12417683, 16559930, -2150200, 11186758, -6692361, -8356719, 10934610, 13948788, 6851248, 2941489, -16296293, 6848370, -5789220, 16268730, 3713817, 16530525, 3844497, 14521808, -6985200, -14670174, -15165039, 4051376, 9173663, 13248444, -16353070, 1067376, -1909845, -1967313, -10641131, -16735118, -7868607, -6982794, 10058958, -5800864, -9605505, 15388107, 5507649, -8374907, -3015411, 3682276, -10670148, 3611398, 9071715, -2797301, 2992698, 3559693, 4166403, 7389120, -9799734, -987408, -8541532, -9330323, 11938028, 7998317, -4047194, 9705372, -1075714, 9843846, 4247931, 14490971, -3411279, -429696, 10032064, -11558997, -14956741, 2500431, -14209732, -3079488, -8702835, 5144927, 3819075, -5775511, 8370154, 1463997, -5298490, -1576135, -8158545, -12807158, 13459217, -16042113, -13708115, -7825061, -14751801, -15792752, 7810028, 11016400, -11776533, 15470086, 7564845, -8957587, 3310940, -6021340, -8173959, 9566878, 13165530, 4446649, 4912357, -15892415, 7764139, -16060671, 1443793, -10761291, 4484338, -1867323, 5602843, 11973527, 11640031, 10576664, -7377466, 7879190, 8582036, -3433806, -14238861, 16657002, 2738885, 6562983, 1978424, -12484142, 16408326, 8496454, -2318475, -9060002, -13178948, 4926960, 5333957, 59789, 13262695, -15112722, -15859597, 2124388, 1648747, 7478683, -11691171, 1869390, 7311864, 11067206, -10651639, -13077173, -9872045, 3956619, 5017358, -16567595, 4192564, -16178360, 2544735, 3421830, 4503707, -2831237, 442533, 16746785, 12112276, 7590092, 13746612, -11354047, 1392727, -10308187, 12474242, 5418891, 4722554, -797197, -4498661, 14544996, -3027676, 13008797, 9388617, 4391376, 15573285, -12194706, 9622333, -6413650, -1699156, -10224940, 4249728, 12024103, -16088595, -4054699, -14204464, -5151121, 14842785, 14739899, -9596471, 2383217, 12980584, 2817312, 2500722, 13119457, 2443117, 1234097]
check_arrays(f, correct, P[0])


exactly equal


True

In [10]:
# line 194
n = hhn
hhn = n >> 2
power_index = 16

for j in range(0, 16):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = CT_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = CT_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1



In [11]:
correct = [14107024, 11483618, 8668100, -3127562, 13522219, 8554446, -16372309, 9578703, 7178533, 12898298, -13290777, 13415413, 11931524, 15882357, 9352796, -8919904, 14342938, 9410558, 10083344, 5718679, -9909629, 4412604, 7879135, 6073218, -2761746, 14076856, 16749236, -5332877, 7488266, -13202350, -13315121, 2378935, 2320474, 3205994, -13214394, 5395999, -1741117, -3055577, -9216921, 2349534, 9094164, -4010926, 1362081, -3657279, 4425360, 1447297, -9144691, -12333948, -12042172, -14157691, 14005269, -995569, 6135483, -10511985, 4691203, -15144578, 4619643, 14730423, -11442736, 9255293, 16073200, -9987199, -328907, 15253821, -8103719, -5649977, 338496, 12845438, -14265720, 1001874, 1100702, -11411623, 12452242, 15285169, 15187198, -7357823, 5019550, -10790521, -7841689, -13541755, 357492, -8017942, -901453, 14728664, -11622347, 15270509, 8996190, 8086905, 45686, 8155831, -13804871, 14414146, -3272783, 4946375, 12574033, 1398128, -14115519, -1589524, -10278056, -3275038, 7899596, 11581862, -11347511, 8723024, -9468765, 12244538, 187595, 16357975, 3979830, 12280423, -2127016, -12866736, -16420887, -15508037, 5882185, 10244696, 14660658, -10244959, -7348471, 12343698, 3593773, -14071022, -1587993, 8075750, -4111644, 9647416, -13992055, 2777362, -6586713, -11861050, 9491264, 5865861, 9010495, -7868800, -6356201, -6249226, -3473250, -3215125, 16530163, -197075, 10145080, 9189315, 6773612, 6713779, 1480450, 6177848, 494877, -2190513, 9735023, -13521881, -13103704, 12494073, -15196409, 13132204, -4664235, -4941500, 3087187, 2659308, 5071855, -5710536, 15630559, -13456962, 6107674, -14008794, -3178304, -3426343, 10069824, -3890539, 4870252, 16492712, 5558734, -6106326, 9748255, -2315411, -7868674, 3854356, 4778924, 13548995, -5337487, -9230663, -10119159, -8154436, -3815828, 4151566, 12957070, -11404897, -4027628, 3742306, -7981407, 9984550, -6702456, 12019180, -8324269, -14831011, 15135784, -4945189, 5155868, -5063794, 2557309, -5644121, -1381486, -3471592, 10045215, -7482260, -12432476, 13715585, 12869072, 14212035, -5810460, -10137331, -9863000, 8462057, -7836633, -16679494, 4303618, -1175945, 5403988, 7795649, 7146578, 9496071, 4912709, 11311972, -14947212, 5875606, 6918193, 10686530, -756580, 13531674, -11228125, 9734386, -15442393, 7888704, 14022656, 108531, -1788013, 3316223, -599110, 1924016, -9882874, -11994826, 2479447, 7145645, 9674083, -142009, -1890450, -5456538, 9855020, 3315507, -6693071, -7269391, 5359850, -6131588, -1598884, -2489741, -5692905, 2067170, -445529, 173774, -12452346, 11789122, -844500, 13600635, -15611173, -6555605, -8580225, 11230433, 10920348, 11658658, -6158037, -12875936, -9988403, 11620407, -6911943, -14570526, -5271870, -1676954, -8725805, -4791805, -12411944, 4531181, 8760867, 11499797, 7801026, -14745814, -9640370, -862324, 16571137, 2180751, -7226314, 11484059, 9429387, 10266929, 15398725, -3939772, 15398933, 8893810, -3257848, 11076814, -15211643, 9816264, 14555634, -763106, -9494819, -6890977, -1017431, 13757601, 8084147, 6531237, -15253829, 9789121, 11010704, -4439024, -531743, -689541, 15488700, 2199106, 6165984, -9774430, 12851953, 6632290, 9482608, 8621741, -706878, 16391348, 16417993, 2515710, -5404200, -10731098, 13931479, 4792261, -2442584, -14154471, -11134883, 15003912, -4891455, -919056, 8127726, -5241260, 377681, 6622420, 3119976, 2435620, 16672151, 13308073, -9830287, 1275574, -204897, 5062748, 15348287, 3308490, 6492017, -8288459, -16187523, -11979100, 8349848, -4396175, -10811368, -12857547, -6098551, -13395081, 6365230, 15020215, -9908912, -3134335, -4287052, -7540723, 139399, -615914, 2046286, -6457109, -14230019, -14101964, 6361996, -13512137, -15981803, -7433552, 9136108, 4150802, -7564855, 85293, 1723773, -7069244, 12280740, 9264536, 15361148, -1039447, 11842095, -15824926, 8918536, -8552557, 1196651, -6130452, 16695163, 15144607, 11319039, 12591571, 5906879, -5659352, 4945965, -8635216, -15235847, 10150762, 13928192, 12178012, 6503322, 5439221, 11395244, -4631123, 15199929, 11351924, 12692517, 10623885, 11800064, 2684401, 14619268, 13665986, 5691713, 2715474, 366120, -4168237, -16592812, -14368633, 3236456, 7327698, -2388235, 7969642, -7089994, -11165420, 2675227, -13710090, -15500303, 6484063, 3827825, 3783267, -12945807, 1917206, 6921255, 267517, 15201895, 7159945, 16032594, -13680549, 15828662, -16047411, 12014514, -41863, -3946031, -2535929, 1203879, -12089048, -5818971, 3616661, -14553387, 7312669, 4127811, 10475932, -6061801, 15816179, 9503679, -1241058, 5053088, -12580917, -9826779, -4709803, -16033310, -4547492, -4282805, -2038260, -1153426, -10740098, 7928492, -10699682, 1903461, 14001547, -5265214, -4555148, 7658450, -5021194, -3637297, -11234686, -12784226, -1271153, -14446392, 8683766, -3681604, -12923955, -2572595, -13852034, -5828679, 10435625, 10811742, -10989253, -14218455, -955445, -6931317, -14095590, 5538871, 6508457, 13510840, -7567018, 15360029, -9265258, 5462266, -1892098, -16282733, 13855320, -7097305, -11158105, -10325506, 5080251, 7967820, -7310896, 3985031, 6892553, 9651191, -14726817, -6175983, 9334210, 14089808, -9867255, 7391982, -786089, 3069402, 15057860, -8784916, -4032471, -10088719, -12567172, 11608908, -15587924, 11385812, -5873669, 12350791, 4861685, 763277, -1453794, 14021025, -15259966, -11694777, -10792516, -12231974, -6978404, 10906594, 12420007, 8573542, 8793818, 14216236, 2863598, 6552395, 3170911, 4189878, -4688898, 11349083, 14312978, -1025326, 10940619, -9644200, 6845436, 11489564, -15827051, -11261929, 1330710, 6169291, -3844264, 2250987, -11138858, -16105690, 4412532, -6585623, -11008353, -12157913, -15038575, 11587292, 5731837, 3282138, 2620602, 6913289, -7426275, -9780358, 1449778, -1463982, 4445659, 1582302, -1741554, -6385835, -3276499, 16100413, -15561880, -10348762, -9363270, 8847829, 6421366, 10130057, 6632417, 3678797, -12969550, 376890, 2728463, 1663996, -2996365, 7113053, -12031862, 14846086, -8979062, -8896175, -7494670, -8628483, 2321779, 3680438, -2108106, 7492149, -2270970, -16507775, 13916255, -5439385, 3566070, 2262574, -16127960, 5018322, -12150343, -12046698, -14402825, -7426436, 184341, 8426199, -16610754, 3735703, 10190853, 9889539, -10138530, 5789923, -15009689, -9664555, -2881773, 2349747, 11995202, -5999133, 7690857, -9833340, 559494, 302408, -7355400, 4037044, -11506200, 1699027, 3739133, 7516306, -11709529, 3614463, -15867480, -11966332, 6909309, -8278335, -7043908, -11065346, -12428470, -3234704, -12150424, 1810642, 12502843, 9522786, -4631181, 15743686, -11370108, 7736115, 15579178, -3756523, 9896233, 3376425, 15099590, -16049132, -13725394, -8169218, 12307834, 4152121, 2995395, -13493263, 407906, 12101116, 545079, 12446413, -2086572, 6996545, -7392352, 3765454, -1686469, 2311301, -4522708, 12656269, -16574533, -14647104, 232467, -16034546, 2789296, -11916307, -7591553, -10938381, -4104298, 4358155, -13578754, 5033634, 308229, -13674594, -14259793, 14120290, 1944446, -13503479, 2514226, 1791136, -7961810, -4859579, 6004241, -4377847, -14884003, 6303287, 9861895, 7465774, 16473066, 3769207, -14670458, 11681187, 4766545, 16635550, 15041952, -6312608, 9313598, 7738849, -416895, -9846736, 6238795, -12640082, 3613397, -6998972, -7877313, -7226426, 14191253, -2343706, -4028407, 504703, 9948504, 4911718, 1018942, -15119028, -4145134, 5537502, 11958356, -15662440, -14708464, -12922150, -11782644, 7617469, -4664766, -12794455, 5623163, 3998531, 1212689, -6853250, -8660650, 15811185, 2034843, 8164257, 2067557, -14572855, -8291960, 12696055, -5075311, 2285760, -9897291, 3257137, -11566923, -6621867, 10030146, -13798214, -5368825, 10575318, -676868, -15974808, -9328874, 6930785, 10095503, 15792365, 2744495, 12529524, -6369709, 15037512, -8776925, 11192007, -15010067, 5291948, 10253113, -7059138, -4567274, 10081844, 10403417, 4909903, -1262858, 12566453, 13132790, -9415350, -1154927, 7132111, -13723337, 8578235, -16524442, -7256067, 209343, -8290850, -15832008, -4857637, -11085133, 13693261, 3932689, 15489565, -15591339, -5571124, -12137649, 6118109, -15151339, -7589779, 1340048, 4489249, 13542076, -3481264, 9734557, 16732211, -6988179, 11289189, 13879968, -16752334, -12537360, -5944809, -6303718, -11507590, 335653, 372460, 321624, 4852399, 11268475, -5185622, -465086, 2629899, -10819552, -6531086, 3936631, -15189898, -16768847, -4857643, -11652682, -16482731, -3016170, 267207, 16366678, 5316791, 10627950, 1891160, 3222614, 11419294, -5047762, 7729100, -16310708, 15468810, 1575206, -13636480, -8250243, -875576, -829349, -213045, 8479087, -1578327, -10644295, 3727476, 7461508, 12307593, 1584168, -16110592, -15281513, 16179020, 9383707, -1486002, 11163049, 2457416, -12892656, -16212534, -6727820, 7529532, -12113084, -9033814, 977659, 13974444, 14406607, -8449656, 16350068, 10947060, -15974447, -11654710, 2583019, -12548420, 11973707, -14408284, 5520366, 6662042, 12352120, 15860668, -2619915, -12324254, 14032267, -10402843, -10355298, 15915800, 11233372, 7048193, -5362101, 14342019, -15941672, 2217053, 6008597, -1504284, 2351225, -12343702, -6224832, -3015304, -799661, -12514744, -9181206, -1950140, 5715797, 3269412, 5440046, -4122307, 10095119, 11860179, -10093324, 15454813, 7326877, 5799955, -16292379, 15063728, 16758009, -12644404, -10870459, 11134557, -1645460, -7801508, -3705336, -11316408, -7845938, 6908971, -14121290, -13338861, -12028498, 944825, 12819174, -985173, -9410907, -16262383, 7304919, 8380399, -9447721, 9840280, -2677149, 12395520, -11940123, 15727251, -9051229, 4476631, -16214001, -7721756, -12895324, 4621632, 5022807, 12894561, -7820934, 9471734, -11732846, 1657581, 2147051, 11891998, -6662009, 3931198, 13134773, 7303797, -6478267, 4179904, -11815855, 12360052, -572919, 3109478, 2034459, -12267937, 5334366, 15229394, -7499266, -9394319, -8944430, 3715119, -9210891, -5616012, -5154447, 146161, -2870381, 8439001, -3759540, -1121127, -10390680, 15872405, 3862465, 2537865, 11908323, 6442629, -5764619, -3873125, 11777133, -2502130, 2793930, 9990679, -9604382, -5084695, 6787212, 14364544, -6654695, 11417916, -16222340, 8187654, 9270368] 
check_arrays(f, correct, P[0])


exactly equal


True

In [12]:
# line 214
n = hhn
hhn = n >> 2
power_index = 64

for j in range(0, 64):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = CT_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = CT_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1


In [13]:
correct = [-14630033, 11707132, -8816911, -8569752, 738733, 10532193, 4350939, 15972044, 1757673, -11538671, -11043794, -8324764, 1461049, 1683481, 16631829, -11587776, 2871108, -15811377, 10302534, 16745473, 7468027, -6280946, 15863675, -7365872, -1320099, -13000482, -11587750, -1849133, 14802379, 5634363, -7795420, 15344248, 3416733, -5812124, -15464743, -6660236, -1848630, -4507505, -14164465, 12909923, 11011821, -332560, -3309305, -14705270, -3298028, -10074172, 13631274, -3510758, 10358025, -509897, -11681315, -7799278, -16334954, 1856628, 14335989, 10629361, 16136841, 10200550, 16455514, -14210176, 8772074, -1077371, 3360551, 7397817, 2933953, 12609738, -10077970, -9117939, -3789186, 16697645, 7347232, 13256169, -3689069, 12740212, -1488653, 2111169, 5679763, 2453171, 5573375, 11582016, -2228160, 15902857, 5162748, -10735889, -9845202, -3092297, 15348768, 9030616, 7403981, 10878394, -15764565, -14171806, 6099349, 11339952, -8352763, 7691061, -14458355, 2215793, 12378233, 8939206, 9697234, -13860170, -16042987, -16490373, 630760, -7438886, -1689102, 10793377, 14768959, 12725167, -2208031, -16342362, -3329219, -14370767, -12965759, 5481610, -15761845, 10539407, -12049458, 1859376, 746530, 6534509, -13511424, -9939075, -13788677, 2365377, -5045293, 10026536, -11471919, 6574358, -7275159, 440554, 14469839, -10236305, 4478336, 10720193, -378524, 15169336, 4096684, 2271233, 4584089, 8149085, 3114858, 10031464, -1434549, 460705, 5861984, -6629637, 2970811, -13210113, -9784458, -4797131, -4631461, 5042928, 10489027, -8646901, 9016999, -1132465, -4587045, 11311617, -8895902, 13227786, 2949996, 14699071, -3755292, -395802, 923301, -6899950, 4411746, 9542388, 4981363, 12872356, 3661010, -9101546, 15576036, -9605979, -11157327, -15374414, 11629867, 3721091, 5106738, 4427972, -9590290, -15141872, 2943632, 2717238, -8767908, -5956982, -11327684, -4675490, -14621617, 14005448, -1334479, 6388600, 118363, 13347104, -15424592, -11634698, -11322101, -8529842, -8762745, 9983993, 3199553, 7726008, -7775260, 3038735, 1446647, 1226311, -3120440, -4262908, -7114707, 1310083, -1485341, -7565964, 2286661, -9118306, -4485612, -13055050, 1720106, -5403954, -14150447, -15665402, -2793723, 13510068, 9855964, -9331332, -15829483, 8971430, 2089225, -6744662, -5402841, 2625240, 7459163, 8590367, -2451512, -2628789, 8268420, 16681410, -12892821, 11608478, -13672534, 2748086, 12063579, 6194758, -11214542, -9356682, 5215291, -12938798, -6711897, 10471171, 9243859, -11487477, 7966424, -8830332, 12173603, -15886856, 15267808, 3935450, -888130, -9306866, -11884257, 6095006, -14169279, -10934215, -11713227, 1802880, -7843635, 12004515, 6547560, -11138240, 6641997, -11707620, 13594572, 278470, 3521009, -9330183, -2341009, 2054074, -4271973, -463578, 137679, -15108162, 8926739, -7787125, -5488677, -11956149, 4287082, 10873070, 15318023, 776264, -12952314, 11001814, -3691139, 4400868, -13868553, 8714334, 1896852, 8351206, 8992545, -10916, -8878655, -1142439, -11554804, -12187853, 6925861, -9917207, 11395221, -6665822, 13331929, -5371810, -299262, -14752916, -8029489, -8426533, -14938075, -10288581, -16298025, 11645280, 2628367, -9268407, -14698030, -16308571, -7566955, 5621521, -369387, -7948716, -9173704, 4483311, -10327849, -1654054, -11266030, -8271557, -3774976, -6702369, -8371160, -3368557, 2632518, -9093322, 16236779, 6467281, 15971782, -15313893, -9540045, 4690921, 3910251, 12446303, 8000895, 3574495, 9996353, -9004128, -13186905, 11756983, -3738615, 15264284, 10041561, 16474651, -12079704, -16691187, 845632, 1989408, -14957410, -9329329, 12179615, 2041102, -424026, -3609831, 10332584, -4539524, 14515853, -9340753, 11906759, -1320354, -16494804, 2763748, -170603, -8151427, 15451545, -11473572, -14919283, 5170913, -5287450, -7777859, 13363725, 14993686, 15375364, 14936930, 10332105, 3197941, 16467889, -15395867, 12301751, 16379425, -1340554, -15158168, -2253431, 13387379, -2608444, 11459317, -6562382, 4386562, 14512137, -9548342, -9041229, -9941575, 4905865, -15731511, 1144552, -10322798, -3977012, 8773511, 1536277, 3237424, -9283704, -9991284, -5027506, -1361677, -4746379, 3135896, -9762485, 9790500, 2415072, 314098, -671778, 5218221, -214877, -3575568, 5749880, -13114070, -8237301, 10987470, 6148863, 14982738, -16640902, 12943949, -960578, -11104981, 4439610, -12509334, 11555223, 10174371, 3905095, -14602495, 7118899, 1532580, -9942840, -4249389, -2396098, 7066854, -1211395, 12783388, 12688837, -907279, 9013701, -16534538, 12471962, 13740267, 10712022, 5328622, 2076744, -15099481, -13698812, -16383327, 16587247, 16733137, 10481415, 1650125, 7236448, 15294549, -16357056, 14773207, -1502605, -12117171, 4703295, -8015099, 7757566, 2514509, 4139780, 6952136, 2927708, -13844927, 6059403, 5760627, 9878336, 8253945, 3601521, -14293513, -16544623, 14590037, -13038503, 16283925, -13703787, -7209558, -14208988, -12835651, -3865157, -14449697, -9340073, 2203884, -12367253, -14006, 15339023, -5571005, -13754454, 15159122, -15933042, 16686263, 15109340, 2435424, -1460761, 12714686, -2044947, -14298275, 5363286, -10009855, 15258517, 11517672, 5751728, -4084278, -5260210, -7230830, -13683445, -6283677, -5206516, 8055846, -15041782, 14397685, 262636, -4149282, -1890934, 11818872, -10166841, -10219103, -10091412, -13621231, 11789805, 11965499, -33652, 11389758, -10784693, 5252418, -8649044, -4838421, -12395697, 3056847, -12777630, -9785542, 3325031, 735494, 13049004, -10557411, -7664357, 11193001, 20902, 10464921, 3380916, 3035562, 451555, 11502967, -9277320, -3509665, 15144185, -2348448, 14694286, 8661162, 6380164, 3347491, 5736116, 8536622, 5132869, -4484096, 7648831, 14476329, -4604260, 8807893, -3402069, -13500832, -4534592, 5388019, 5153540, -15008031, 12064580, -1001743, -10961653, -9497011, -14332916, -6012432, -353219, -2541414, 905747, 5471393, 590237, -704769, 4097655, -9995024, 6309261, 16237780, -13051502, -15454614, 27432, 5272950, 6595499, 13430980, 567676, 7610890, 9328239, 4997989, 7810819, -13899146, 14812474, -13776132, 12559515, -9901235, -1535001, -3429417, 3321728, -15811294, 8398623, -11185191, -3042947, -15726510, -6954344, -13592021, -16420037, -1195178, -7411406, 16154227, -3952305, 13509318, -5114184, 4285870, 2891613, -14575786, -7077552, 1056939, 6800498, 1906491, -13792736, 12710959, 9203006, 6373052, -7843062, -10198600, -16539662, -2450094, -1617979, -938484, -16115651, -9261202, -2442643, 5436853, 2052206, -9453918, 13113316, 9828968, 13200946, 8690751, 11665619, -15800032, -14892854, 10716398, -1855053, 7271135, 1958836, -15255475, -10860554, -15223669, 8666763, -8031005, -11888828, 8701207, 11509823, -4519220, -3797775, 14990360, -9190764, -1327471, 14550449, 12147999, -10084812, -11446704, 5811441, -3541639, -9170819, 1445637, -15690650, -7915721, 11504542, -10240219, 12154294, 8719323, -6697752, 3665195, -8222856, 9038955, 12858029, 12186497, -2236531, 5768143, 5145707, 12364393, 15379832, 10826458, -2793726, 6264120, 41004, -10811514, -7705316, -16597252, -6783557, -7071582, 11018395, 15119897, 1222610, -13107002, 456459, -12432621, -5099571, 5217545, 9317220, -9583457, 4935664, 12712116, -7417501, 11267170, 7978033, 1841900, 14846078, -15879662, -3493239, -9317873, 10967607, 16043451, 3753953, 13830037, 14595679, 15186512, -8835572, 9307372, -10418008, -9658020, 8054270, 12707592, -5206611, 9332436, -11123719, 8587355, 11413591, 3812783, 2530197, -13165610, 6229840, 2756480, -5751210, 15942748, -6953568, 5788206, -15650092, 11079435, 8878655, 962631, -14035115, -962495, -12714779, -11242534, -15741403, -3567036, -6766521, 9342453, -13791240, -7730169, 16262909, -2592476, -7060321, 2133122, -13805183, 16385694, -15123130, 8950646, -6321822, 13194694, 15452902, -16070954, 10926965, 13132672, 12173651, 14985149, -3618018, -60362, -15968645, -12345573, 3651809, -4023852, 15065725, 3612630, 9790666, 986683, 15328980, -7809569, 1393234, 10157937, 8124984, -13927887, -9932623, -1906222, -10395050, -6388185, 16200638, -6700195, -2414519, 8364090, 4362903, -13007164, -337308, -9391029, -9868315, -10933850, 6629311, 10277101, 13419539, 15076502, -226389, 10415151, 16098453, 13817658, -10518788, 13389867, -12177638, 8595237, -4962035, 5963536, -14149133, 1421132, -16168038, 5437204, -7387153, 11781345, -3067796, 6806409, -4784199, 7826032, -9399400, 7601895, 11246958, 7958004, 16155211, 15047441, 2414234, 7271452, -8186265, -9926977, -9107781, -2252178, -11562693, 7962312, 13182976, 14080029, -6268105, 201154, -8141063, -6974435, -6363524, -10301169, 5134696, -6487378, 6318576, 11669451, 11935453, 13198557, 2164864, 1718141, 13925430, 1338729, 8648514, 14733461, 3240583, -839040, 12916079, 2111888, 3596691, -13332235, 16013161, -883629, -12088842, -5887461, -13616314, 3571707, 4669987, 10462083, 12259392, 10572842, -11981318, 7152922, -4826575, 2269130, -11899626, -5088487, -5794889, 12349377, 16066804, -7986841, -4120891, 248159, -14748387, 4109616, -3913447, -16291219, -13208838, -491623, -2814116, -15512163, -12192378, -7100025, 15596108, -7962297, -8848607, 8986703, 12281045, -13677541, -7319939, -13457674, -13805402, -6356955, -13695135, -599973, 9745988, -2219558, 5181556, -4445005, 8353915, -4228887, -1609930, 12233176, -4277520, 7199424, -12648169, 6030227, 8771433, 6154942, 4946770, -10532795, 4213281, -2295340, -10348629, -14553567, -1213370, 16338461, -16399145, 6058475, 7753909, -15259454, 7317778, 13510910, -7260248, -8540949, -11442538, 14888030, -10995586, 9628527, 14609452, 9708436, -451043, 10062050, -7359074, -1556673, 5320230, -8734138, 10862223, -13190121, -6387175, 13235562, 14250568, 16232008, -16000606, 4776708, 11604950, -4139793, -1453067, -1508436, -13375043, -13396731, 15501888, -15283161, 1280166, -8244549, -13641456, 655685, 12312010, 14048873, 13610290, -10512124, 15507659, -6969175, -6626256, -1039497, 859458, 8875752, 1803155, -15694224, 12823788, -9490554, -11536092, -1296973, -7842429, -12746932, 12332254, 15803311, 9019659, 10068507, -15963575, 12054646, 2425198, 9966851, 12548643, -1996143, -2566312, -10870339, 12458749, -13483653, 5203633, 4281012, 16726699, 13917011, 12995318, 10180671] 
check_arrays(f, correct, P[0])


exactly equal


True

In [14]:
# line 214
n = hhn
hhn = n >> 2
power_index = 256

for j in range(0, 256):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = CT_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = CT_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1


In [15]:
correctf = [4007053, -8142000, 9611364, 3104125, 10198393, -15537891, -9706515, -15549392, 14630722, -1886723, 9464306, -15177613, 7104679, 15338530, 5778310, 11173014, -11299894, 15030334, -3003782, 10757774, 5329279, -182356, 16402316, 8322869, -12878783, 12316500, 14362534, 14469690, 10787860, 14045346, -15105920, 15931893, -15184014, 6300379, 10188490, 12362077, 821780, 15340238, 14332149, -4338350, -7467609, 9350086, 13879882, -5265412, -6548180, -11180552, -92707, 4629327, -503362, -5104900, -3972535, -16087777, 16578507, -15786878, -11312685, 12281914, 1074806, 6187367, -15360840, 5545357, 2592085, -14525923, 3755799, 9715998, 11648312, -13068465, -828058, 13984023, 1376820, -15428646, 5457125, -6562043, -4078444, 12237505, -13760853, -9154484, -9345049, 8015647, -15154416, 5652533, -9131409, -5710487, -10761172, 16690428, -3282016, 14295185, -1940484, -14903156, -5344331, -12651402, 13969755, 91565, 13883112, -11206826, -16338597, 4509370, -10239456, -16548922, 9798237, -7292942, -11129435, 1968138, 13184825, 1215071, 1865612, 11767769, 13915173, 8524823, -2976845, -7177420, -3264471, 5393898, 347827, 5918601, -12242060, -7341244, -14503754, 7824576, 10760659, -28187, 5145641, -2936620, -3412808, 4189907, -3413505, 4906101, -14435655, -8661312, -2212149, 5252154, -2012293, -13365051, -16130905, 5901779, 11610660, -10602852, 3402581, 15294805, -7101387, -13110095, -1136830, -13619705, -8116009, 7658563, -12362177, 8750964, -13255163, 11128180, 15569108, -12092823, -15315651, -9827727, 3231767, -12165573, -14832470, 5240432, -12245196, 11062160, -14200901, -15648741, 8600710, 14135136, -14808835, -9960282, -15052007, 6865900, 14177091, 12538185, -13769914, 13559645, 7918137, 9939116, 5281077, -1964889, 9493702, 1834150, 6058491, -11592732, 11477580, 16528027, 9532620, 1387158, -2084048, 11591222, -11777047, 14616563, -3640846, 12575858, 5009868, -13958723, -10585550, 7774006, -6855059, -2072883, -4245545, 7835571, -7590793, 15252416, -4325923, 2066606, 410175, -7177706, -13664804, -14618645, -11867780, -172269, -11095720, -7965271, -15030042, -14383275, 9920900, 7010657, -11624892, -4267457, 4216634, 5734351, 15238336, -5756045, 7884024, -1758426, -8200333, -13417715, -3107655, 1674252, 11342085, 15229202, -11178045, -9519723, 3490341, 7704356, -8163727, 5325930, -2771796, 2214095, -12342129, 9186145, -1055557, -4277066, -11251363, 16107329, 1532490, -2909142, 11862579, 1924611, 1841072, 3116276, 5471155, 11814003, 12084112, -2450723, 550055, -3480695, -6296894, -16533678, -2754318, -9650088, 3535475, 2035590, -6496200, -5104307, 12109143, 8557103, 2614840, -3717440, -3013494, -2410725, -4338628, -3539724, 3056111, 12963457, -14852036, -8527629, 4375140, 812623, 9327084, 6313104, 6611725, -10629029, -11780305, 6433573, -15961466, 5691887, -2791849, 13612144, -6973589, -11888627, 6064029, -9156521, 14248818, -3639134, 14964062, 2148009, 1654828, 14279327, 15092367, -12240741, -12000087, 755669, -13565581, -1152930, -1127379, -4977147, -14950052, -14460042, -13581501, 8438194, 14543361, -15246947, -3786291, 7952733, 12849689, 2761248, -4723844, -4258161, -9928327, -13207624, 4722098, 1975550, -6521433, 1732359, 3488401, -8553881, 137130, 13236904, -13814465, 15165747, -485605, -2343225, 12993890, -16683735, 9113225, -13184439, -7997812, -3941684, 13486431, -16646839, -5165170, 11046486, -15119276, -13782305, 11777468, 9872611, -15975639, -8887986, -10327854, -1679962, 1014625, -6916142, 1861856, -16061952, -2991589, -9923577, 5298144, 5146270, -4236656, 12388119, 3324349, -15929133, -10051308, 7887613, 2471971, 12426594, 9009502, -16637033, -13492909, 12024333, 16462251, -16689779, 13574881, -2350254, 15399464, -2111016, 10783564, 9943833, 14690204, -745806, -3265847, 2429963, 2759422, -7218032, -4712286, 511028, -7305632, -9642910, 2579853, -8343514, 14160489, -13996046, -8777366, -13082446, -9414984, -3504659, 4572859, -11687816, -10234722, 11987463, 14780466, 13812884, 15293593, 12779955, -13247490, -6553678, -1739351, 12488393, 13561519, -5050459, -2188567, 13300967, -1238523, -8056768, -15066156, 8453399, 2707446, -15100171, 15868224, -7059978, -8449343, 11326107, 7406550, 4281507, 15081985, -6982282, -6523822, 8084407, 1633848, -13365154, 3407851, 7463947, -15972050, 14924183, -1808329, 3457329, -1226266, -15633088, -11827622, -4326295, 15920430, -6150677, 6142101, 1846586, 6164073, -9820487, 4809264, 14467530, -7598856, 5108394, 4988702, -8719263, 7383493, 4122911, -7317602, -9034382, -6333238, 13776593, -13652722, 8713834, 13279964, -5648882, 12360364, -10693695, -9513446, 10825420, -4729141, 15722593, 9781777, -11960305, 3815810, 6738041, -5360168, 10521776, 224419, -3713577, 14803882, 4740892, 9921624, -10653218, 14463545, 15305983, 11274190, 9065739, -15221797, 16765843, 4797466, -15654237, 11172844, -1982496, -914835, 6130571, 4900299, -9878130, -14433474, 807630, 11592016, 3811962, 5510885, -10650141, -12785807, 15457279, -8650089, 2168662, 2678259, -9051303, 12318627, -11690165, -16499041, -14147484, -14004474, 14469168, -6470790, -4457925, -751492, 5837163, 9138599, 699187, -10469331, -11465648, -9751527, -4698760, -14072020, -15571455, 12279141, 980612, -8784957, 7892098, 5770412, -8112020, -16351179, -7924004, -6788512, 7696272, -4599552, -3871944, -5838009, 12004815, 5622950, 14945270, -3036951, -9324695, -707085, 12934123, 16290391, 6466991, -9163994, -14639227, -12081702, -4045664, -14349629, 12916812, -13500593, -10332731, -14247079, -10374592, 14365071, 2714021, 776222, 15778631, 12512260, 15467201, -7221877, 14598973, -3410322, 10821125, -1918749, -12015988, -8602968, 10826455, 8114783, 15182386, 12909966, -15645317, 3900780, -14184290, -11473194, 16717969, -11655303, -12006512, -4833506, -1313819, -700573, -11290470, 12381667, -6285984, 16436721, -7824421, 5515280, -7988952, -4786852, -16520803, -5172635, -3603329, -12280083, -8871302, -2651113, -5793931, 6788508, -15503181, -513278, 1280114, 7633122, 6494708, 11165436, -4443910, 8074138, 11586332, -7896038, -13617339, 13772290, 11503706, -8204471, 10930152, -7634568, -2941891, -15865995, 4554340, 9324556, -4152905, 2891129, 2443059, -5240489, -49544, -8683494, -12722838, -10014548, 3603504, -1235715, -3133678, 16284353, -8010247, 15506551, 7537562, 7831107, 15768718, 14485037, 2417293, 2437364, -14099565, -3566563, -11211996, -2247393, -4594655, 1779879, 9824569, 14932108, 9191870, -12834360, -15600528, 11742892, 10220080, 252019, -9823167, 7947033, -8146457, -14556607, 13988083, -2436893, -11641993, -6109354, -5898600, 1687859, -10118103, 9248566, -14654726, 15950624, 15585661, -15846651, 14893236, -2872962, -6065502, -7873428, -6380560, 15892299, -15643286, -14407997, 7811322, -9156389, 561964, 3150959, -7464832, 3072933, -7657938, 7772716, -1294919, 12771785, 3996182, 12436870, -8593647, 5238939, -4744088, -8493535, 5777973, 10875466, 6906935, 14995453, 794977, -7505685, -7625832, 4924700, -11002606, -10418488, 7550270, 6476161, 12223196, 12459096, -3189462, 11223861, -5470949, -16144034, 10555138, 13525008, 11259586, 11585592, 3596260, -6449335, -2601879, -5329518, -14279165, -13494068, -5936902, 2188671, -3155985, 13026022, 281614, 11092179, -4657159, 15306663, 13494856, -10148853, 13259466, -10569051, 14147333, -4964850, -12586388, -8816456, -4481908, 2030654, -7266815, 8585264, -4424837, -5248774, -703604, 7378690, -6248033, 1866080, -4329994, -3365819, -4517457, -4541763, 1480500, 13545981, 13228381, -6001250, -10652324, 8260278, 9304212, -115303, -6903690, -6116717, 1819618, -9575087, -15177845, -2868014, 10770368, -6278192, 9336052, 15556237, -2939240, -8650670, 168735, -8606212, -4131129, 16743936, 7929119, 7040117, -11208134, -4373145, 13850215, 7766776, -4881958, 7452344, -3729008, -9208495, -15191482, 3990930, 15119981, 9367630, -2517345, 9028427, -734445, -8306802, 7631939, 5713829, -1812872, 15395109, 6074179, -15093966, -13213096, -3619343, -12028086, -5800834, 15663509, -3735978, 3359670, -554169, -119924, -10581700, -11094848, 1267001, 12379684, 4214847, 2894041, -13373760, -3393204, 12187845, 13602380, 5630575, 780305, -7568465, -12933320, -1196115, 14664807, 14273742, 13093613, 15467231, -10189805, -13367892, -8339332, 11271328, 1911081, -16331934, -11536801, -1563866, 9584461, -12019661, 16722776, 11324027, -13598620, -1506144, -12788134, 12498864, -10475770, 15993234, 1078330, -4202252, 16633762, 2780938, 7553321, -5833130, -6980959, 3527433, 6184904, -9570810, 663750, -1096466, 14834241, 781699, 6330428, 12781536, 11700844, -6569333, -9435130, 9962708, -4636686, 10325449, -7555230, 9652333, -9804225, 8732667, 16693529, 12059461, 2287013, 1869053, -7556071, -9669076, -8676063, 13323417, 6065441, -4218621, 5373379, 8895703, 8063518, 13015230, -8973612, 4432652, -11522300, -10737702, 11943198, -8867875, -13252540, 4664589, -7935957, 5718786, 13039813, 9818596, -2805141, 16680125, -9449543, -16744100, -678446, 11088766, 16704561, 2020187, 10394417, -1959674, 6611843, 3458707, -10716271, -198420, -8197804, -2518842, 11767540, 13912348, -867173, 602836, -15297892, -14245418, -9326105, 11949604, 15750897, 11462486, 9961193, 5597495, -4030541, -10610534, -12627691, 12351801, 3932688, -1907030, -8943844, -2050976, -8116558, 11114337, -1081480, 9252524, -2731296, -16736063, -6895245, 9003982, 7804788, -3029518, -12243857, 5077603, 14590363, -15696721, 12881879, -7969670, 3444394, -1649593, 1321389, 13115162, -5665902, 2229296, -12213257, -1279416, 5670377, -3541353, 3659737, -10793041, 4793541, -7022342, 2589835, 11567512, 10668921, 3092124, 6417608, -12806304, 14619234, 13185267, 6282723, -8543431, -6533687, -3405503, -7066079, 12414331, 13358710, 1553032, 9322514, -1028997, 4816715, 13731220, 10219131, 8566795, 12833876, -9399119, 16455663, -10725844, 6393466, 10041944, 6825284, -10963964, 15104994, -9160397, -7640147, -15021248, -14157493, -10198518, 12872235, 779628, -3670116, 499256, 9603852, 11864911, -5603862, -11287517, -7567563, 7465978, 16520510, 7955769, -16163578, 9568678, 896799, -3866401, -3352702, 302959, -11402795, 15182896, 12561175, 9409545, -7028519, 5635518, 8268115, 6984544, -12795373, -7951780, 13568731]
correctg =[1073351, 12831593, -16475839, 3323843, -8303473, 9043234, 11374047, 11115235, -627613, -16713452, 9436548, 11860434, 12486666, -12976446, 6971523, -11859095, -12901541, 13167282, -3775431, -7016081, 3339197, 7102982, 2856597, 12910549, 4263547, -14497049, 12076282, 9727939, -5756866, 6898272, 10645051, -14661073, 8891878, -3920889, -9964110, 3101047, 9735771, -11375979, -7295745, 6762430, 2746353, 14440020, -6163280, -2190091, 2841339, -14868731, 9770204, -301217, 7898393, 15420821, 6183942, 10527790, 14115112, -7727584, 6855419, 11747364, -7218158, -2366134, -8451427, 14926108, 8898832, 3473267, -5200404, 9100335, -5579599, 1740029, -2165566, -14488297, 3811176, 7373755, -9676855, -8221926, -1791751, -5189033, 741113, 13935090, -9233204, 5992395, 8053879, 16282232, 9732922, -3187788, 5565400, -2667853, -7209464, 16312893, 2450514, -6966503, 4285068, -4962333, -16606591, -11894071, 9875252, 7120244, 16612971, -3086062, 4625872, -3626507, -2609540, -5751149, 4376256, 4994996, 16032490, -16501862, 8465753, 6875571, 13836298, -6350131, 13436024, -8802675, 12189614, 13225895, 3391538, 670404, 13685654, -730124, 4925697, 12473935, -7182370, 16557363, 276376, -5312979, 16698136, -6078695, 11023329, 16072702, 16260355, -3002045, 11682130, 3077670, -12208602, 11958507, 898094, 4300287, -4804824, 5613737, -16674956, -7014065, 16336116, -5329513, -11055946, -11658292, -8672375, -1843976, -13346780, 420060, -4350521, 4836806, -8111532, -14888556, 4441873, 2331671, 10103851, 9987504, -3881187, -10412334, 16164372, -2133554, 10630743, -2040799, -9822011, 12050872, 13012017, 9858125, 601070, -6240201, 3590597, -13862112, 11316963, -15607271, -1429444, 16449774, 2040313, -14915971, 14995632, 16125107, 13075694, -15484805, 15892713, 7997988, 13151059, -11393100, -5814812, -6875864, 15848974, 10812306, -4272390, 4646649, 107306, -12393180, 3688164, -13855164, 15992987, 12518231, -3097632, 7061610, 315375, 923850, 14012482, 4329474, 7159522, -1732685, 6194631, 182276, 9565215, 12422219, -7883956, -12420209, 6008957, 7594008, 12727903, 1204004, 438778, 11624648, -2667420, -6729398, 7219984, 5974564, -2617219, 213624, -10277139, -3213639, -7849859, -14543055, -4243232, -3137952, -13575333, 11239040, -3056636, -16030190, 7778552, 3242368, -8199550, 9752783, 8632472, 7393703, 208593, 5125980, 9836822, -8296492, 11363979, 5362184, 7406799, 7147015, -10627701, -14667884, -4620764, 15599345, 3902032, -6862629, 13799753, 7209174, 11189444, 12110343, 5094665, 1061135, -2167925, -12981780, -8253007, -10499018, 7508090, -3601484, -15356882, -1460368, -6968395, -8116327, -10361895, 994575, -8293653, -7725976, 13003638, -10200203, 8646753, 15139620, -11328688, -65717, -13307451, -12778714, -5275887, 14363194, 15793428, 9937062, -14851643, -1599585, -3095735, 2585264, -3218416, -5077526, -9484750, -4277911, -7241147, -12757728, -10816079, -15200533, 9184184, -14856583, 4275430, 3918760, 4530439, -12766821, -15579041, -8138666, -12989126, -14293101, 14475078, 547850, 13937780, 8196870, 2540894, -6341070, -9655425, -1592508, 2712816, 7167194, 9284495, 15055698, -11293762, -5483703, 15026748, -11811829, -15127873, -509000, -9237813, -14722275, -7998818, -16267064, 5557035, 56920, 7225347, -7188312, 14981892, -10249670, -13102021, -10912913, 2515349, 9579257, -6761568, 3100630, -5412575, 8869323, -6207662, -16705875, -7892489, 5068531, -9116788, 11106097, 8591818, -10426272, -11290216, 2083647, 12122904, 594145, 6964747, 15651833, 4158044, -8808433, 2182534, 11344508, -10361929, -16046691, -4059835, 12360910, -11728544, -3115248, 2361481, -2513432, 9302193, -8170008, 6514113, -14452592, -6701775, 14043588, -15433248, -5288912, 7981700, 14099721, 7477989, 2828157, 13487688, 9621942, 9397489, 4002120, -13446834, 7139232, -3678790, -10756662, -198718, 1790257, -1741592, -6030532, -16486294, -626687, 6625909, 5146547, 4605134, -2888046, -7352783, -14951752, -1144800, 537918, 9639743, -3664597, -920068, 15614372, -11760765, -11136330, -11133849, -16257572, -3574947, 16249080, -11172697, 12358859, 13610765, -4027033, -12381123, -6227586, 220667, -3581202, 1520811, 4721368, 11191814, -13669122, 3432550, -9023632, 1849642, 9732644, 13503910, 9744355, -3268641, -1248089, 6468197, -8989982, -12237538, 6394572, -12881788, -10980297, 6127047, -10675497, -5714131, -6844516, -12834002, -995630, -7529227, 792079, 6191536, 8676673, -331386, 11562816, -5687276, 15228375, -11868629, -526766, -15603530, -4594594, 12992275, -10137158, -7587896, 2388007, 1736549, -1294247, -1472923, 11958034, 4573620, -7738765, 9968836, 10647041, -16479320, -10703920, -542855, -1907508, 13964682, 11859032, -12819838, -2424124, -4085847, -5145603, -785225, 8192956, 15623165, 15860948, -14127196, -2290255, -10198720, 6401872, -5195815, 686884, 392779, -764290, 3830588, 16340993, -6507579, 4727473, -514913, 14648052, 424916, 12059166, 1322224, -2900142, 7945131, 12910037, -10596478, 10495286, -7789031, -15253773, 14025894, -3737001, -4136181, -3270359, 9810060, 15893268, -1299621, 39391, -13124146, 7980116, -11570655, -14865911, 6782644, 6613880, -2590111, 8528176, 671799, -11597972, -6742945, -4746215, 11173579, 5143352, 7343324, 5984009, 15406435, 8105953, 9741382, 11623957, -5361398, -9249334, -6318412, 12602517, 11951134, -10059583, 6459163, 3600160, 10147546, -2301843, 11466015, 12735345, 4736477, 11358044, -7600921, -12519015, -7774860, -2425276, -9556862, -2659900, 7864017, -7392687, 5078832, 4120642, 15059804, 6995520, 9762812, -7050242, -14974171, -8026106, 8098007, 2952074, 16402944, -12758651, 5635982, -2343457, -10221084, -6507445, -8258309, 11257433, -110374, -9659191, 1163816, -8338931, 2902554, -16859, 4410618, 11984617, -2834626, 8220005, -6172117, -2119655, -7929675, 13026661, -13212298, 15688151, -499030, 4292559, 5731786, 4901792, 3075380, -4695641, -394542, -7692931, 9746086, 15072338, 579904, 824187, 6431934, -1648098, -12009705, -1748770, 4632299, 958758, 15135704, -15190351, 15689876, 14581262, -8721035, 7264161, 247527, 3518935, -9881031, -1164093, -15267247, -8443194, 7886041, 7552872, 13130022, 9541182, -492757, 5711504, 7568373, -3925883, 9755791, -3908086, -5809840, -4208398, -8267238, 9114414, -16333797, 16327529, 1912797, -8673411, -4068771, 6266418, -3228105, 104680, -5701221, -13900285, -13656788, 3415032, 3022591, 15965558, 5336945, 3570407, -16344255, 13351084, 3937054, 377634, -11396365, -11025340, 10476724, -7729386, -923511, -9909559, 10710627, 16582526, 4106915, 386896, -4301664, 7839034, 4654520, -5342944, 16211296, 5754434, -129300, 9414252, 618536, 3288483, -11104178, -7962437, 1264262, -1055544, 1650063, 7802778, 16061578, 12050256, 5222115, 14442761, 1124981, 1842292, 15044079, -7297301, 1630549, 3846084, 14221981, -4094578, 11630830, -11945922, 15580444, 166476, -15599557, -5660007, -792576, -10486348, 9545774, -7872285, 3758904, 11496505, 2556049, -9518513, 13710377, -3063349, -8663322, -11188641, 16078375, -489818, -1335568, 14407127, 10262940, 12038356, -8631963, 1724379, 247075, -7216243, 12636775, -7230252, 5994734, 13316315, -4037040, 13296412, -11005841, -14483990, -12741281, -12392180, -613630, -15301151, 4144370, 9059485, -3397500, 9772449, -3360008, -6564538, -3682802, -1602439, -5915160, -4504836, -11119014, -15918024, -12568646, -3876139, 12091136, -877561, 16533166, -14295264, 15368666, 1015264, 6948790, 536667, -3356105, -13838993, -4218757, -803013, 6209998, 1829456, -15488449, -7953596, 9673824, 9621197, -7378413, 13233274, -2735672, 12100346, 15470144, 4330587, 9368087, 10212496, -9032563, 1112519, -1418414, 13703148, -6807515, 3804181, 507550, 11403854, -4009109, 1137838, 11276793, -10330732, -15237304, -956670, -5988826, 380814, -14965250, 14519564, -12980006, -7566436, 6799344, 8711505, -67942, -1706218, -3803382, -11462197, 15206026, 5456590, 2272122, -3669644, -5640334, 15121025, 8169004, -14396015, 11736808, 6925422, 13070219, 12266754, -7041666, -9469646, 1912026, 321635, 15555478, 14735578, 14477842, 10250884, -13398592, 26011, 15981864, 1713394, 1510250, 5219165, 4815682, 857536, -3195209, -10778879, 36431, 5327479, -13478154, -11447457, 7614799, -2864360, -2484744, -16083733, 5461534, 2422664, -10326034, -12489207, 8964311, -13205203, 5466013, 3617123, -6256645, -3830414, -10693980, -12100194, 7093572, 8625418, 13587888, -8362083, -14044948, 8012564, -12915545, -649898, 14646284, -15501208, 4426859, -11432710, -7661312, -14101987, -9364223, -1545489, -2299171, -13218494, -4615567, -4730905, 4145162, 7280660, -14689531, 7984146, -3149324, -14183444, 15103364, 2132719, 5873560, -3816093, -11266639, -11522229, -12940316, 4678882, 10652515, -5290081, 15661301, -10521775, -11257695, 13136420, -14173080, -10579725, -13910344, -663650, 16064145, -9864562, -11035364, 2417075, -70043, -2091172, -13278605, 6639626, -8095634, 5010394, -4197464, 207800, -7481674, -6170084, 3719361, 1407479, 13323619, 8391236, -2484348, 9394573, 2105024, 9234561, 7089937, -11897615, -12660139, -14397142, 4577432, -902523, 3020848, 286015, 13905188, 11703323, 14552621, -7532929, 9635977, 13709131, -3089737, 1963405, 2201595, 3995987, -1955246, -13483410, 1909409, -12056061, 14759977, 9171569, 9009462, 14385188, -4124849, -13730174, -15823440, -2340135, 15219863, 7292785, -11204298, -6126982, 12494811, -5116189, -16560417, -8736766, 7973163, 12202997, 14431661, -5497744, -6566565, 7717913, 1555243, 6048736, 11585653, 14739442, 2054545, 14437493, 1572116, 8727429, 7617785, 7836834, 571485, -10608310, -2875758, -4065067, -12014483, 16356247, 3539298, 129166, -9670354, 13669623, 2926233, -7604890, 7199295, -58841, -8711367, 15284326, -286117, -11362503, 3039923, -15712484, 1999879, 6126159, 2141101, 16297776, -8735413, 10414233, -12497555, -8089655, 3145928, -8280295, 13031775, 11406570, -10560840, -3970015, 8657154, -6790902, 15098729, -11691923, 15027520, -11391617, -9309826, -6608178, 13671672, 7484395, -2774017, 9618551, -4580559, -13540197, -11398732, 10430032, 4242228, -402608, 2992570, 15735589, 14695217, -6085700, 13142821, -12861414] 
check_arrays(f, correctf, P[0])
check_arrays(g, correctg, P[0])




exactly equal
exactly equal


True

In [16]:
# line 253

n = 1 << logn_top

Nf = [0] * (n>>1)
Ng = [0] * (n>>1)

for i in range(0, n>>1):
    Nf[i] = mod_mul_standard(f[2*i], f[2*i+1], P[0])
    Ng[i] = mod_mul_standard(g[2*i], g[2*i+1], P[0])



In [17]:
correct_Nf = [-4216416, 3246891, -22775474, -11648082, -9581527, -7218100, 12667452, -9815119, -14546573, -819718, -6939782, -14418513, -33228658, -8240369, -451752, -6811874, -14623550, 13916411, -3491751, -12011823, -5771879, -27798555, -3493234, -28657622, -7637030, -8088854, -9977509, -17567056, 5577010, -21956873, -24615813, 9581319, -29235244, -18805154, -5413496, -3810099, -27816302, 11710395, -17924272, -15456317, -13687979, -24614131, -33330138, -14666049, -9696961, 468113, -25826126, -4249531, 9687946, -17226053, -20572481, 11278053, -8110703, -9396239, 4426168, -230248, 551707, -9206000, -33038954, -15648753, -22441653, -26747771, -14077085, 6484830, -31965509, 12278362, -9222949, -25799905, 6352133, -11999982, 10011672, -1492691, -17903359, -5004979, -7618647, -14289618, -32991008, -1974202, -6007655, -4759801, 9601448, 7910062, -2095482, 3340589, -4840042, -11756704, -13475397, 13767941, -19259927, -5995231, -11328524, -17782612, -14551209, -13485902, -10195537, -30371385, -6716198, -2984574, -14062024, -21428970, -6386126, -2149662, -6292949, 5343489, -58243, -2341605, -9360688, -1380692, -22903148, -3833106, -8206559, -11337100, 13052000, -13734398, -332463, -13507308, -25570917, -33429616, -14142643, -13361135, -28692683, 13454954, 13109187, 7587800, -29394087, -23707320, -4606319, -4649841, -4583609, -4131788, 8128676, -5220927, -16248134, 13098270, 15840962, -3473704, 10180330, -3146484, -3367975, -12554516, -3915053, -13538627, -7040082, -3752153, -3746969, 5939371, -7711714, -12115462, -8007832, 4703040, 4896148, 4195036, -19003700, -27659031, -1026477, -14392815, -14955444, -3380580, 2400376, -24622100, -32809851, 2762809, -24868016, -11453567, -7692529, -10818459, 11540772, -3962299, -77614, 14131140, -2751505, -10773628, 13233683, -2740841, -15918636, 13247044, -8350280, -30957147, -24257489, -5752987, -5305371, -17687494, -8035042, -5413030, -27352657, -16510285, -4122192, -11985389, -24861029, -2583073, -1101496, -3098989, -15713004, -23277178, -249301, -11874663, -7684497, -2733858, 16108174, 16745386, -152519, -2867011, -23413179, -6895228, -15206539, -28630903, -220568, -6498555, -20397326, -12505821, -15325628, -1828543, -24242435, -996968, -20754070, -17194639, -2769022, 15740222, -33499093, -8558023, -30271528, 6112492, -9186242, -27698485, -11164783, 12142900, -6251663, -4073694, -6484424, -1979113, -28283021, -1672202, -25262992, -15229718, -1467661, -6865683, -8724186, -6848873, -4623424, 10178489, -17383092, -18556803, -18299539, -18836180, -1793673, -9767655, 14391117, -17649791, -23685385, -18733097, -4822801, -12711835, -8798941, -22537831, -14839154, -8935794, -4961963, -6589918, -12838424, 12396268, -8733660, -3709623, -32517481, 2702885, -6084351, -15667589, -13194936, -9117393, 10155377, -4448288, 9185957, 14770504, -6175399, -6379962, -10993381, 11747092, 2350315, 2238547, -9895309, -19759484, -28771122, 7573960, -18567348, -1551301, -64071, -21630291, -17181854, 6095352, 11332728, 4538527, -16573966, -22698518, -18658167, -1172002, -16266750, -1434861, 15006493, -20014714, -4553884, 14708077, -28949731, -2537753, 326183, 765567, -23248358, -4914540, -30253876, -28518095, 14026686, -11271027, 11070921, -16741915, -13436433, -26079748, -861154, 6837168, 4187150, -27756212, 12473155, 6963903, 7751740, 3049341, -15767641, 16134953, -12457617, -23222727, -18641161, -7920799, -16319659, -4470389, -25425144, -297440, -13445682, 12370135, 14250034, -16136873, -26166772, -12902980, -9960939, -23369343, -3630678, -5735706, -16366679, -32194380, -29511512, -14705623, 1597415, 16630898, -1405786, -8858864, 15555679, -22445801, -2323209, -9620322, 4573578, 13129437, -13052107, 11737165, -11606092, -2363701, -16587398, -17024832, 7949093, -16156055, -29982409, 12779080, -2180105, -9324974, -18223608, 5036221, -8887319, -19792125, -3025443, -22230971, -12820803, 11130819, 10308860, 872408, -32457052, -3227297, -5785611, -14628200, -7615159, -29840928, 12753774, 5603398, -6826096, -4734346, -26665510, -5606641, 6092478, -11597445, -31175160, -28013672, -20951182, -8910497, 7792653, -10598218, -7712244, -23849399, -2430842, -5054441, -5679130, -12492849, 10453300, 12053873, 9738846, 14993877, -9866616, -8434465, -16025951, -25218872, 7912405, -2729553, 16065515, -31879791, -16511716, -25112279, -7471686, -1832230, 3640099, -17821506, -11795210, -7853318, -10270232, -3027898, -3964369, 15831694, 4947433, 7878314, -33005890, -32657218, -20649223, -2681465, -11101868, -18313983, 15040629, 4953030, -31396983, 7206817, -2664558, -30668097, -28405566, 3091376, -12095701, 2131499, -947952, -8106201, -8687915, 14169224, 13422734, -17320793, -8673880, -12719091, -25510975, -13320037, -4689174, -4382223, -6171168, -14888104, -8979145, -14045532, -8753688, -7299392, -6350704, -21693455, -26600235, -14720047, 3839323, 986422, -5055650, -9466265, -32648939, -22200124, -23849315, -27743147, -13938637, -26072072, -17021602, -8815306, 3610609, -11656335, -26216322, 8302982, -7231067, 10173868, -2319338, -15665184, -27550182, 5198998, -7708994, -3674047, -6973143, 8563558, -7292545, 5018608, 7242617, -15814584, -18256140, 1422431, -4127096, 5760756, -10711060, -15208336, 11167232, 13108475, -1820526, 1805172, -3412041, 278926, -1329814, -28522770] 

correct_Ng = [-12764401, -27755635, -32299850, -13335780, -213648, 12583759, -20265741, -5304153, -22571491, 536997, 13194989, -5955834, -3327139, -9121127, -789773, -22559299, -32643970, -17603247, -3115010, -20040055, -11946039, 4724955, -17665354, -16627639, 3860344, -11938525, -11726686, 14142174, -5492048, -27110043, 6415253, -17720217, -1748659, -5922199, 9955581, 9399809, -14322657, 11619830, -4025422, 10777032, -12607698, -997187, -31251237, -28056158, -32706077, 12811297, 11422954, -19486795, -18333375, 15963609, -5354004, -24485363, 3554619, -11312698, -29658994, -7222808, -5697138, -28223397, 11938701, -25424601, -15834962, -21756090, 15864801, -19755466, -7904232, -12498124, 15560234, -23534116, -653494, -15472464, -3030009, -16117039, -19342415, -22461211, -2449540, -2572117, -15196315, -4314467, -28424330, -18463955, -20125790, 13025241, -4139818, -24358221, -18590015, -8656510, -7248919, 934319, -32652194, 15244145, -20996058, -13004658, 7540265, -25512818, -27865411, -1398240, -13823993, -19739249, 8067242, -15828705, -26371494, -4031579, -7000375, -1355103, -14226740, -5804508, 15616371, -4038917, -15699990, -17975888, -4144653, 4677433, 3196348, -30307435, 370538, -943222, -29346355, 7867375, -5694050, -20926935, 13894560, 10178634, 15242148, -5775300, -30035915, -14302195, 5394490, -6207720, -7034265, -6852891, -2846703, 9835263, -10065618, -24703335, 16756497, -33324516, 4478725, 5411266, 12991272, -19187017, 16575101, 8543847, -27109375, -6576133, 11098212, -692788, -6315704, -28291005, -16774597, -32541584, 16467331, 6889112, 12526958, -7159529, -7728059, -12743559, -14041958, -7428671, -4118398, -16950772, 16612804, -2148508, -14696975, -6145036, -13197033, -26231358, 12506969, -6510967, -18251595, -7186568, -1756120, -16996090, -770429, -23840364, -32950429, 8698235, 6507928, -22692936, -5229021, 7863027, -2978730, -264198, -31794122, -29747730, -15473174, -18300798, 5954158, 2438065, 2303605, 15583546, -15549983, -5608450, -14731399, -22067315, -5408884, -13050498, -10440577, -15536646, 15301967, -25641102, -19545205, -29713896, 1082407, 16236719, -9511568, -6895303, -15555367, -3925560, -8464036, -533944, -14670437, -27849493, 4091380, -11510888, 1649534, -218772, -5314826, -14137374, -4556877, -5543729, 3162114, -2439498, -7076130, -4904543, -24563579, -14425125, 15450055, -11231968, -15655638, -19808310, -7395122, -7508791, 843545, 10532636, 2654172, -6455854, 5016900, -4348624, -3205013, 14974589, 9882964, -31832631, -5092085, -22196771, 9845570, -19901673, -21138506, -10389954, -1588393, -365088, -7511064, 392830, -27818230, 5437656, -5618188, -29106886, -32837245, 10535504, 8251749, -18043139, -10579107, 1540897, 2736702, -3429497, -11051816, 11603464, -16746951, -28062259, -19120084, 13288684, -11154845, 8799800, -12450087, 7857686, -13891739, 12096988, -24538394, -14314053, 15267076, -26878491, -1895970, -7659125, -18038706, -20953995, 1471924, -13890200, -5511415, -9538054, -17766540, 5009696, -31497641, 6048507, -28435648, -27819723, -27351331, -2403061, -10165692, -13954777, 10103323, -8762854, -27284360, 2803883, -5686070, -21669522, -11983986, -14640766, -2425449, 7932466, 8123489, 9480282, 11268451, -13557283, -32947789, -1460574, 8994537, -25947199, -22545734, -645752, -7678791, -32587473, 16727025, -11023179, -10155531, -15864220, -15903189, 15816418, 2146129, -7112762, -5782515, 6988484, 5025520, -16989605, -21459337, -13343384, -23006903, -11160411, 7504565, -19514353, -750084, 3168932, -2622551, 4978614, -7871958, -571924, -26091488, -17417607, -6558126, -14857913, 4069460, -2150880, 15460366, -20652173, -15802493, -15706690, -5579718, -9167907, 10626162, -964242, -21230371, -5320929, 6796619, -23524744, 3139739, 116770, 15726995, 11216120, -12277582, 2793405, -5173885, 7356902, 8985730, 12036497, -12719002, -19823258, -1910766, -775294, 6529348, 4986603, 2335137, 13528810, -1592233, -21653172, -3540114, 15358100, 13932018, -1668968, -20233062, -17344970, -10940866, -11445800, -20130636, -13003428, -11076452, -13369152, 13473798, 3185286, 7449021, 7154476, 12929673, -16214002, -24291727, -26712279, 13983750, -14994679, -15725352, -3194700, 3931617, -8464923, -23426093, 15182745, -5132856, 10349233, 8246731, -3312096, 3115855, -15425559, 5447084, 14101864, 3472423, -26674381, -5803838, -1923462, 11467604, 12133610, -33302013, -28146444, -4197935, -31332746, -9840357, -12389892, -3278610, 5297024, 8316066, 14075647, -4460135, 12916796, -11032603, -10460816, 5959513, -32094043, -11413350, -24381870, -17397014, -4008851, -10282309, -30953339, -12867549, -8810946, -19869313, -25687470, -24908211, 10905913, -3681865, -1850551, -29608680, 7927675, -9904071, -2756261, -16914441, -13988041, -9588886, -13428281, 8954546, -889830, 9181462, -1641022, -25101802, -6616376, 39424, 13268517, 15622892, 4785289, 13371141, -8985589, 1892402, 3614456, -29667997, -11266418, 8230744, 11847524, -11342418, -13747734, 14586575, -11245124, 3722854, -12440813, 73506, -21059018, -9949966, -7162133, -8970497, 11165888, -32570931, 172471, 15783068, -15248967, -12725645, -25684883, -947754, 11325495, -31491852, -26659487, -9535985, 2802824, -6298087, -11328996, 14996868, -31124876, -7924965, 7674684, -10950821, -8215948]

check_arrays(Nf, correct_Nf,P[0])
check_arrays(Ng, correct_Ng,P[0])

not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True

In [18]:
NNf = [0] * (n>>2)
NNg = [0] * (n>>2)

for i in range(0, n>>2):
    NNf[i] = mod_mul_standard(Nf[2*i], Nf[2*i+1], P[0])
    NNg[i] = mod_mul_standard(Ng[2*i], Ng[2*i+1], P[0])

In [19]:
correct_NNf = [-28149806, -11549349, -9800741, -32435338, -3996419, 6707278, -8633855, 7243271, -12877703, 12178263, 5528514, 15586252, -5740326, 10620243, -2790280, -32607106, -2609664, 13716592, -30979997, 13785581, -9775323, 5464858, -13559504, 61942, -25012185, -7784308, 10095462, -24843289, -30425592, -12773485, -15163524, -22123543, -2917253, 5693177, -21856705, -6079442, -2282106, -325128, 13811223, -11901478, -16648809, -505196, -7276271, -5586779, -719825, -3819233, -10960178, 12569941, 5585632, -9853810, -13853237, -8796093, -19890, -232707, 149561, 6302852, -33286128, 1082491, -13714124, -9517214, -1042795, -15817652, -169081, 15153468, 6433132, -27167535, -10060131, -33040786, -27658633, 7606022, 9443455, 9563640, -29477659, -555732, -28618018, 809928, -9571962, 2664805, -1558564, -24250400, -11064412, -10917637, 617344, -20735286, -13783430, -10899243, -17415355, -32506785, -5388682, -10485901, 2007113, 4617874, 11630533, 16138173, -8531158, 12050153, -9387045, -15725306, 8429125, -5423392, 12108588, -15233975, -9516319, 2231589, 15502752, 13364351, 7269694, 11574501, -13058543, -10875685, -2628966, 2093121, -24814814, 5793499, 1329368, -12826885, -8699311, -13037117, 6718631, -23032612, 339174, -689346, -6934585, -33524244, -8761706, -14451613, 15250404, 1916993, -808917, -4766857, -1470947, -32552210, 1476247, -5728935, -22639241, -5202405, 12759335, -11884839, -6155361, 2576454, -9549248, 10849519, 10604202, -24626551, -15971813, 7505893, 3940811, -8124769, -30183459, -23794715, 13644649, -217530, -16321440, -5580381, -30272873, -17751596, 10439468, -5280731, -15647353, -14519372, 9972012, -21105767, 13308076, 3492207, 16270851, 7569538, -11688669, -32186349, 15126218, 7953164, 8714790, -5091750, -14329289, 14179264, 16745063, -9631736, -8043970, 10401975, -2875563, -7640657, 15543315, -24269632, -3536086, -1658836, -29730410, 13927143, 7259568, -14701527, -4951677, -659703, 1224110, 456456, 6041473, 6580662, 16543514, -1045025, 1522500, -4274789, -21176077, 9262659, 11455878, -15058823, -2167421, 14951633, -4138177, -11145718, -3918292, -7657916, 15695962, 1280637, -20404993, -6040784, 11473102, -27934849, -8845158, 10926052, 1030038, -15493903, 8615905, -14366480, 11341739, -15008943, -16816127, -12635454, -29576769, -2362979, -6067053, -9223611, -4902969, -2982962, 10935682, -16658804, -10340553, -13822943, -2918591, 9581241, -15319365, -1371663, 16243418, 2390401, -19466116, -2625640, -17096362, -11221240, -7544877, 11783318, -11709974, -6010573, -3227200, -4436992, -15132222, 12005679, -6242709, -6400311, -18088624, 14423137] 
correct_NNg = [11705123, 9031327, -5788011, -4798667, -25603863, -28988821, 4589380, -13463038, -5211951, 6891218, -13640511, -4215120, -23878843, -8299199, 5449801, -28352354, -8839975, 14004083, -23746103, -6201339, 15412527, -1391263, -26238188, -6495160, -27176920, 10448304, -4484983, 2416899, 14089337, -33376227, -12918066, -24606106, 10223367, -26494423, -3323485, 9847272, 14643725, -14960061, -1875621, 10408077, -21889143, 10809611, -10377568, -29771308, -702247, -12257691, -3601232, -10886852, 15676461, -2045456, -8671395, 5578223, 3517959, -1864361, -12761092, -12587713, -394928, -5732907, -21931394, -6618963, -13604379, -12237954, -8124685, -19794338, -5790811, -14508356, 3250633, -5849508, 16679182, -18208959, -6765028, 6695814, -10915103, -7204226, -10628558, -10213530, -22711012, -12401079, -11206406, -5961179, -14976960, -7147123, -4092593, -23872059, 11161654, -11432488, 8053821, -10251904, -6518777, -28863404, -15746469, 872655, 11646800, -2689564, 10698722, -14197179, -11902093, -4310962, -2516785, -15802269, 4419822, -1828751, 5188068, 771311, -9806927, 4720728, -3690452, -4427476, -7871763, -4123524, -10195058, 10708713, 6089213, -1621909, -6099206, 856216, -14259383, -18159237, -1856295, -28886494, -7807035, 13687224, -19564924, 16590562, -14351461, -20433992, -10011581, -8331828, -9144694, -16534090, -10698767, -5825166, -30176872, -8514627, -26188016, -3298195, -17016111, -30654063, -7406375, -14847455, 13063888, -6110395, -11779696, -3890355, -13451954, -25786696, -4121568, -5077007, 14383020, -6625099, -3735740, 1117952, -6950913, -15471214, 13228933, -32018342, -7052705, -19460778, -14617160, -9451386, -21587648, -12581942, -11871833, -4718353, -1694467, -18095040, -8398620, -6401831, -12771500, -24464849, -18675372, 8434825, -8106093, -10446377, -6255207, -17180905, 2457562, 2684978, -19430415, 3592516, -28221204, -10759706, 932024, -30854526, -1718671, -12454145, 680710, -9715031, 6279824, -9978633, -1643853, -30104116, -31159574, -6125443, -4293070, -2279617, 4025291, 7092795, 3952258, -13962245, -16404111, -19894434, 10665655, -8027612, -6425376, -1818586, -31500410, -28506126, 4397684, -3766010, -10185550, -12271008, 3496343, 9783816, 8773217, -13613308, 3833043, -28251280, -9114611, -31700675, 7912821, 12039578, 13464299, 1131071, -4333585, -3416102, 9418981, -4318259, 6488818, -1203238, -12331605, -19931702, -28042519, 1971258, 14710041, 5050373, -29501968, -4091889, 12343835, -16761633, 6849509, -12164941, -29621238, -18728602, -16071416, -6652609, -6814569, -26950025, -7123151, -23871820, -2067643, -12952612, 3987807, -17509816, -7856958, 7516137] 

check_arrays(NNf, correct_NNf,P[0])
check_arrays(NNg, correct_NNg,P[0])


not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True

In [20]:
# line 294

n = 1 << logn_top
f = NNf
g = NNg


hhn = 1
n = hhn << 2
power_index = 127
for j in range(0, 64):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = GS_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = GS_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
    power_index -= 1
        

In [21]:
correct_f = [-81935234, 1477708, 21725340, 16347901, 1320275, 15557131, 19565505, 18397954, -13135011, 20436706, 32802135, 10585946, -30517469, -3505544, 5026791, 26410631, -6087488, 33005677, 11392484, 13343118, -17808027, 5322158, 18215721, 28429056, -47544320, 6772690, 26339912, 17931847, -80486144, 3867903, 9296973, 4954823, -25160223, 1342637, 6364042, 10784837, -697489, 12520545, 14109720, 30872329, -30017055, 9484553, 1282781, 24971749, -2929295, 6315833, 22751526, 8053188, -26917508, -8408749, 16057804, 302347, 6199816, 25958535, 16848118, 24889320, -55434975, 8993805, 14315692, 23825561, -1876060, 16359844, 23765179, 12490660, -63835320, -13191369, 1082972, 22722124, -34595853, 12506799, 20140435, 10210149, -57841481, -2088202, 28149373, 4479401, -32716121, -2497470, 19293651, 28932677, -42099991, 4597809, 10915132, 33447323, -74604813, 18835979, 4602641, 17112896, -9249596, 3428050, 6326519, 33784694, -2262636, 516552, 2915139, 31928652, -22106618, -4994127, 6179081, 11900673, -10410117, 13166805, 3410141, 15245401, -19389376, 1987424, 7987757, 31711294, -24470073, -2665635, 27236902, 29163761, -30518832, -8450866, 19889541, 22192995, -38050409, 15648393, 30393298, 5088954, -40809001, 8490890, 33750204, 20122030, -39596259, 6471151, 28208922, 22556712, -39598931, 3517527, 22699849, 28931293, -32094334, -1928200, 13225156, 24945686, -2704411, 13135540, 23426248, 25010958, -12722078, -12900062, 16835990, 19083202, -12649878, 8325510, 30121849, 25318218, -40551055, 12698274, 29855256, 5036993, -69926290, -13171802, 17900985, 19474878, -25007988, 13126327, 32772873, 14162199, -27883809, -4073547, 3921522, 6333661, -53584966, -13598972, 12727054, 12752662, -6847915, -11517877, 15622540, 25428041, 6963302, 10459933, 15862983, 5771363, -8158215, -15228443, 1482951, 2955185, -13921239, -14455700, 6151011, 5158984, -23245226, -6964010, 2686909, 11412966, -3930814, -13320198, 19022921, 15237752, -5429713, -8458428, 11198133, 2725620, -14665707, -5628417, 24634373, 28316737, 9181267, 32159009, 13370224, 27043857, -26860103, 5245419, 23542984, 4028150, -43019515, 8337951, 7116759, 27268537, -14380853, 7430725, 29099113, 15631559, -20214440, 17149959, 17539651, 25191719, -33118785, 9246163, 25286362, 8858166, -47230412, -3054000, 31109806, 4266847, -13609053, -13934812, 26095079, 1570589, -17500846, 2492698, 2640407, 27594440, -31607546, 11467532, 17228705, 25296498, -50409358, -9369738, 2005614, 20638025, -13482106, -1935341, 20212861, 18834628, -10790735, -6024143, 14271995, 29027556, -16308507, -4155693, 8129125, 23197598] 
correct_g = [-23400565, 2121549, 14364508, 31484099, -63466342, 8111100, 6030138, 24749897, -16176364, -3919843, 5475275, 20058488, -55080595, 14734078, 18850028, 30872100, -24783334, 1672870, 7142207, 7990970, -18712084, -14157480, 31670976, 19072850, -18796700, 9599768, 14722771, 11728250, -56811062, 7338776, 13059028, 27370241, -9747269, -12118798, 31992949, 1293613, 8216120, 224261, 17433248, 5861769, -51228408, -1814745, 25370565, 22602717, -27448022, -2673926, 487461, 5359212, 10537833, -4017459, 18320104, 1728290, -23695207, -3419795, 11169060, 16645842, -34678192, -1306495, 12451289, 25390590, -53761356, -15944217, 2729166, 28780972, -22898042, -15180099, 2259608, 12233133, -1598991, 19535493, 15884940, 5342125, -38961417, 8852637, 31557690, 29438641, -52279676, 17007855, 1578475, 9755810, -50088735, 3971486, 11787600, 15112255, -2468917, 2129253, 10731856, 28191996, -50255995, 20293742, 30892894, 9876026, 5458779, 14012970, 12686824, 23769197, -34532109, 14265403, 30101941, 19409994, 8550450, 27135867, 8533478, 1197468, -13204127, -7182694, 15829027, 23547104, -11481632, 6262965, 350005, 18225699, -775686, 22495337, 10628888, 10931983, -63161409, 4074819, 10770736, 2349537, 2905827, 11677862, 22025543, 32960326, -53128862, 11892104, 17381671, 12835394, -42202717, 232627, 1641916, 8001519, -68177710, 15269449, 28751206, 29468988, -69924004, -6710716, 25215878, 528046, -8716558, 2385540, 20650347, 27237407, -48437225, 11940453, 6811574, 28169307, 5140133, -4974528, 20071260, 4277112, -41211536, 10715329, 27207412, 470027, -50582029, -7020715, 13820682, 21873314, -50759776, 23448014, 7791162, 1586190, -34589958, 8169386, 18230560, 8289222, -47476896, -12459574, 30524985, 25064513, -41988582, -13979216, 17476981, 25900097, -10695359, 3452379, 21007436, 3873633, -68903412, 11868353, 13954471, 31505742, -23207137, 3475770, 4394206, 14452435, -35446778, 7401759, 31699233, 28234975, -43857704, 10345709, 4912773, 21596403, 1108099, 7586989, 20864015, 27671850, -33660502, 2021817, 9927700, 25522667, -68250498, 9655423, 15228049, 5303049, -21824884, -738692, 19577136, 16504308, 8440068, -1964360, 33494207, 7571828, -65233523, 10320520, 23350023, 13413668, 997432, 12312832, 10436366, 23736600, -2648965, 12301095, 24373470, 20467441, -26977727, 3661076, 15968800, 13370176, -39861184, 24870317, 20930794, 32140687, -38011655, -5081022, 1765880, 2878506, -53665272, 17087242, 5108069, 9884235, -56488619, 5686364, 16545114, 25111609, -46015226, 14346158, 8076664, 17860316, -13862830, 5427743, 14733236, 25160927] 
check_arrays(f, correct_f,P[0])
check_arrays(g, correct_g,P[0])

not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True

In [22]:
# line 318

hhn = n
n = hhn << 2
print(hhn)
print(n)
power_index = 31
for j in range(0, 16):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = GS_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = GS_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
    power_index -= 1

4
16


In [23]:
correct_f = [-124267439, -33134673, 12019097, 4641758, 9908901, -9591392, 3643521, 10829530, 10683027, 18539648, 15551705, 30761843, 23486589, 1781030, 9053451, 1955981, -151925979, 15418091, -1855584, -2441830, 7356454, 13711762, 12960349, -943867, 22674781, 28656000, 16388005, 20963048, 7062765, 8818850, 17420898, 24953308, -58804062, -3886769, -22592605, 7581429, 20177843, 17369138, 4341799, -4110329, 25876579, 22156349, 10059742, 29677413, 23587628, 28351145, 28008483, 1320991, -78028727, -24197239, 3886119, -5592786, 24223549, -1599378, 21911818, 9338491, 1883486, 16926052, 12813502, 3726202, 29508784, 11315885, 11578834, 32519535, -188988775, -5270242, 1565757, -756323, -6221663, 9243412, 3408672, -8251352, 19899508, 20897681, 19456704, 2133046, 5995709, 21986653, 18865682, 30661832, -128217036, -6171947, -8790906, 15622554, -13416710, 14776286, 11394467, 6011526, 24539089, 3750798, 16677561, 32785965, 15302325, 32395056, 4869250, 28009750, -76376184, 7494467, 11263544, -12629882, 1338565, -5969157, -3119164, 5118963, 4296250, 1430430, 23131822, 26083523, 30657162, 33652004, 13770660, 17998971, -148974501, -11390769, 11590954, 2860017, 10143039, 944486, 702979, 26647785, 28452243, 18044595, 5812665, 22527110, 12743589, 16429814, 11650270, 30587835, -87119754, 1824805, 9086569, -2679872, -7354507, 27835567, -11695449, 2102080, 1524462, 7494933, 11210596, 15223289, 7349545, 5402885, 13743113, 33336410, -148135211, -12572028, 9999952, -3108386, 15239005, -8054815, -731143, 14662606, 32736220, 8533766, 24998813, 17493957, 243626, 16281608, 27335688, 31963557, -81353388, -18730463, 14583762, -16814947, 12644154, 21358361, 13404300, 8324430, 14543424, 29542060, 28607233, 2846540, -25807, 27968764, 29144298, 1424236, -49255494, -49968351, -4206545, 1214550, -11757499, 21102270, -6279251, 188061, 11301505, 6080261, 20504975, 9649679, 11037739, 232197, 21890631, 32884476, -37774256, -10232754, 5645040, -4986310, 21899699, -3300843, -7130311, 8915710, 27299496, 31372628, 6580301, 32963318, 27591396, 1736339, 15835931, 17325406, -110733593, 8614461, 11941211, 9849307, -1483330, 17250857, -6410389, -683991, 25924591, 17550452, 27807709, 17471416, 10013134, 6236214, 18100892, 29656864, -109947857, -3028582, 9973323, -8372300, -13378722, -10145273, -4105301, -10861243, 25825703, 10066590, 19581241, 10410920, 26738026, 16819684, 30622628, 27649187, -90990706, -21484915, -22481079, -8953204, 14355203, 385080, 11381590, -5151706, 18065898, 30637752, 5113701, 25938616, 19598877, 24530921, 25313154, 28543678]
correct_g = [-158123866, -12503453, -22380725, 6513573, 7147234, 5438034, -5942268, 10500761, 5883301, 10482268, 20608867, 17223154, 9162411, 20832989, 4512953, 13273197, -119103180, -29096403, -505692, -938363, -1522409, 17573471, -736534, 4050870, 24777910, 2191936, 4647810, 5125160, 2457123, 4851764, 25140667, 24755993, -80207579, -16383208, 8183549, 1566974, 6559115, -6018122, -11871875, 1815686, 32945151, 22786387, 9292674, 3329301, 21599719, 7675563, 1562083, 19163649, -101596922, -24687966, 11119282, 5445020, 12661292, 13603501, 12656206, -12696100, 16077097, 30176421, 29503336, 28955198, 28251510, 32353232, 7766657, 32236639, -115738126, -3334451, -15819961, -10330965, 1991124, -4444561, -2373929, -161911, 27817584, 15011208, 1358172, 30579568, 23386572, 15135275, 16999932, 29651091, -97354868, 6857114, -1001500, 9848800, 12495989, 10491065, 4785954, -1661454, 24455921, 20869476, 7399410, 25085576, 8424225, 33106509, 17165981, 15867840, -50667418, 6931204, -12286223, -4720409, 16456731, -9261595, 3745473, -12615273, 10404851, 12010463, 30357597, 21262915, 10584801, 14691173, 29871148, 13375236, -114160130, -16960552, -6293836, -8023434, 12395425, 19646125, -5431570, 22493066, 1492017, 1040434, 33183143, 14613241, 32076591, 27633619, 4724536, 1346495, -189020989, 11176900, 9158673, -1864714, -15949705, 12089368, 13179333, 16505175, 17589722, 5558544, 23102521, 13604981, 16617401, 24734751, 20019752, 7454717, -135090657, 10660539, 810254, -12310914, -12011352, 8253385, -5095054, 17394917, 26909402, 30408584, 18774650, 31814796, 28305187, 13075473, 26837701, 6393732, -174815212, -28371727, 6923014, -6260652, 694780, -4455520, 168539, -1409868, 19146366, 19141476, 28568913, 1074202, 30574103, 8766367, 3070138, 17279685, -138252686, -7352076, 3954672, 10966111, -14444749, 15110867, 9945357, -8943501, 21608482, 9599268, 30553305, 27461857, 27576801, 2737970, 31156438, 18401538, -144660605, -3940399, -16168137, 12993295, 26433604, 5782877, -7792718, -8424430, 9744983, 6849985, 31596283, 12292988, 10042226, 20985832, 18524232, 16872971, -77620907, -13620037, -13793279, -5874270, 1788036, 4647494, 28319777, 10501849, 22009151, 2420471, 2803288, 13324926, 1559173, 1413489, 10529338, 31298271, -107499531, 2201129, -4061730, 1756136, 21035714, -3335731, 15527856, 13930530, 16140192, 31357082, 16519531, 33178691, 2502599, 11955120, 24773532, 1905652, -170031947, -24553167, -22637591, 10916413, 23264823, 3227764, 12458684, -7760255, 8843972, 23920982, 1535547, 8619618, 24906816, 16996430, 14401607, 86561] 

check_arrays(f, correct_f,P[0])
check_arrays(g, correct_g,P[0])

not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True

In [24]:
# line 346

hhn = n
n = hhn << 2
print(hhn)
print(n)
power_index = 7
for j in range(0, 4):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = GS_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = GS_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
    power_index -= 1

16
64


In [25]:
correct_f = [-413026207, -45800590, -8542973, 4188571, -5433927, -13660207, 9307150, 15113825, -5982801, -14372962, -12287720, -15522505, 16545092, 16716573, -1039008, -6350859, 20414498, 6038464, -16484812, -8045867, 18946424, 25273642, 21423835, 3292158, 12246435, -13076720, 5675133, 2474121, 16375874, 14560571, 11997868, 573180, 14774511, 10580574, 23077137, 15842913, 5585872, 32726036, 11860926, 26726986, 18262858, 24171228, 18940882, 22764387, 25754914, 3090441, 26264177, 15961946, 17436785, 9205368, 15131790, 31190292, 27901334, 27493917, 20844310, 4971443, 22822853, 2081491, 27760989, 4117206, 9650857, 30014042, 26072018, 27181742, -542556496, -15338491, -17920988, 5096366, -8156769, -14555310, 12386954, -4023415, 10086416, -22977170, -2021922, 16428970, -2401889, 3812516, -17944812, 6607377, 21448937, -14319485, 1161722, -2135025, 2101811, -13899354, 2511863, 8281872, -16627011, -6992828, 1799123, -8468536, -15859322, 9351940, 9713311, 4975247, 21658845, 5270123, 32539896, 11326243, 20879864, 5318005, 19369919, 25125169, 31223733, 8550370, 3450921, 271122, 24521002, 20248992, 16494721, 27909908, 5868191, 2070496, 32895398, 13397000, 31446879, 2084517, 9985855, 12570528, 28889414, 14649337, 33236539, 6223281, 28938898, 17537247, 14327041, 4934267, -365863847, -79446037, -4086599, -21388655, 8771153, -4859291, -5301543, -8273160, -6995063, -15449654, -15329394, 11663128, -14945234, -17215220, -8537281, -1042332, 250998, 14753845, -15847369, -458350, 14096287, -4256533, -14200381, -2867385, 10141653, -2572208, -4245322, -15098371, 1645159, 10003698, 20887198, -16009879, 20223002, 5561765, 7984952, 12631347, 17026168, 28658422, 14543588, 18377221, 32385782, 13395345, 7293589, 28943255, 24039839, 2719798, 26606788, 13589830, 17592003, 3240984, 8787978, 19937050, 18234297, 24629445, 29277457, 11522267, 1753160, 32866738, 491322, 27684455, 30035171, 16410252, 11589619, 25501332, -349446412, -26131790, -28471842, -12462507, -12157487, 4189821, -6264411, -7781230, -3535323, -11023589, -8017722, -13866741, -16709578, 15772821, -10778406, 2524124, 25523226, 6097497, -3213743, 2213398, 4341444, 5635675, 10999328, 14563135, -9525638, 10728116, -2587883, 19695078, 28401786, -8134589, 13550121, 12317380, 23185281, 16111667, 32756470, 31184008, 7144763, 6781774, 20581672, 19338802, 11770996, 2930749, 29053886, 8679986, 2467868, 12158716, 18002661, 6351379, 32703414, 21905154, 15309352, 22757128, 5817117, 4596012, 8320663, 7998943, 27068828, 28488996, 18422456, 30184530, 18187642, 27989440, 25353888, 27334927] 
correct_g = [-459031547, -82671030, -37133923, 12587204, -8705105, -2953453, -5894471, 3671217, 12582785, -1463662, -3047987, -12467861, -5629911, -1387126, 5432023, -11221533, 98209, -2190393, 16612465, -5521639, -4203088, -1562081, 1076896, 8142112, 1201118, -10961010, -7191837, 18866104, -6549705, -14048865, -11957018, 5798691, 7595303, 7263515, 11194974, 12296458, 12137146, 12676637, 6793411, 17240526, 26884003, 20842094, 33111251, 15414241, 29720725, 22956792, 5176969, 31346723, 19848918, 30681532, 10674076, 13506563, 14854103, 28803883, 31067007, 30911923, 13361653, 2431420, 26070375, 29648702, 22249188, 1738626, 11973586, 26675535, -377920542, -6506685, -35401520, -13226008, 9788932, 16431034, 725928, 8054428, -2930301, 15381244, 5197648, -9109711, 7371515, -10084435, 1660923, -6860012, -1320817, 10877253, -2404037, -5202897, -6489666, -7263865, 23579955, 4604959, 3290250, 4815832, 3440289, 19000307, -12630670, -4906217, 7853805, 9668042, 22082273, 18768647, 25692129, 1743952, 25261623, 19400623, 28910022, 7872338, 32313293, 5855473, 13903312, 7329589, 20703056, 25389878, 14578933, 3697785, 32124698, 24792181, 10492347, 6987963, 5976148, 13816353, 6144555, 6736156, -225378, 7939673, 30755329, 31269900, 15069026, 26610996, 30047848, 8163556, -637179544, -47436701, -12703724, -9470169, -41711026, -2552237, -15352162, -10003614, -15397039, -2392802, 348378, 6855162, 2422481, 15764224, 13983355, 15979335, 5324048, -6625625, 16311394, 2953480, -11120634, 10003181, 24083766, 15813700, 9646282, 12300648, 2982085, -15815949, 10861795, 17820294, -15569754, -5627577, 20877286, 6151964, 9434462, 26740951, 5665248, 23190962, 25943274, 10098351, 20439738, 33218094, 4676117, 2106277, 6988581, 15056079, 31254502, 14034030, 17491136, 23599788, 12825496, 2019351, 3609211, 460220, 16722387, 1831915, 10858258, 3862962, 30463182, 18901104, 2802945, 16307394, 12568877, 25813835, -499812990, -39912474, -56660737, -13758763, 5421503, 10322404, -18587075, 8247694, -10362376, -2552154, -14646025, 315549, 5460477, -15749803, 1128035, 16613118, -3591533, 9141759, -5183179, -3192549, -10157118, 5167146, 5959572, 7548584, 16016820, 14459190, 11282045, 6960822, -11140419, 2775358, 16700350, 8142699, 16287123, 23054207, 31642804, 8847442, 10327573, 26202484, 32672200, 13071355, 17801344, 5795495, 1498748, 27712932, 12477449, 1592910, 6219545, 10547911, 18237811, 27297651, 29037555, 27110614, 12482487, 30679652, 33039779, 23923930, 20731054, 32748036, 14629132, 17559606, 7332758, 21184092, 21597715, 12714153]

check_arrays(f, correct_f,P[0])
check_arrays(g, correct_g,P[0])

not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True

In [26]:
# line 362

hhn = n
n = hhn << 2
print(hhn)
print(n)
power_index = 1
for j in range(0, 1):
    for i in range(0, hhn):
        f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i] = GS_4pts(P[0], f[j*n + i], f[j*n + hhn + i], f[j*n + 2*hhn + i], f[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
        g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i] = GS_4pts(P[0], g[j*n + i], g[j*n + hhn + i], g[j*n + 2*hhn + i], g[j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -mu_omegas[power_index], -mu_omegas[(power_index<<1)+1], -mu_omegas[power_index<<1])
    power_index -= 1

64
256


In [27]:
correct_f = [-1670892962, -166716908, -59022402, -24566225, -16977030, -28884987, -23422187, -4963980, -6426771, -63823375, -37656758, -1297148, -17511609, -14463647, -38299507, 1738310, 536985, -20980016, -34384202, -8425844, -27614708, 12753430, -12815692, -10280557, -3764561, -11913640, 641051, -1397708, -2986840, -7768717, -10952176, 1855928, 12740965, 3973792, -4292556, 3883837, -16464007, 6383563, -744569, -11082833, -7007642, 15497355, -8361396, -6441924, 9682949, 4667610, -13282664, -3287611, 6499719, 2871665, 5023844, -13369541, 16298953, -8296783, 1327611, -30037493, 13433581, 10985888, 12810632, 1108798, -13838443, -8700030, 10241892, -15698743, -15603352, 7699431, 11705106, 11048470, -11869969, 15260725, -14087149, -7309457, 7055002, 21383641, 5558881, 8383477, 8418292, -9733177, -6328801, -4667683, -15114307, 3859531, 7654130, -6304724, 3717834, 3097378, 5956000, 6820987, 1906122, -15717452, 18551858, 5020946, 336739, 9357050, -13726388, -2631790, 29314008, 1443702, 16819825, 15286823, 7878970, 8651720, -2731676, -184951, -7933420, -3715777, 12267714, 28582739, -5707476, 24323825, 12890875, 4941445, 7458649, 13653663, 14612592, -12748808, 15416903, 2744799, -15272197, 3441381, -1558588, 681375, 22980834, 7563590, -3528930, -298896, 15022878, -2640192, 19196936, 21463574, 783416, 21969369, 14884185, 6342039, 5789173, 12769504, 27098559, 5718696, 1293228, 1355548, 24728653, 8970240, 28152391, 11389598, 9804705, 27593060, 16372357, 22649980, 13669912, 15213628, 19027432, 32686034, 21678699, 31438222, 16262748, 3262441, 18793731, 8167205, 30644331, 26908906, 10625537, 6136555, 30625095, 32517237, 1533179, 25712271, 30578848, 5837482, 31542703, 4232331, 6066906, 11044126, 7053751, 21588852, 3102621, 19066099, 20928436, 23916493, 16539274, 30431035, 11161378, 5212313, 9132664, 16784284, 7189725, 20632634, 3165810, 17161685, 2497567, 6214610, 13939374, 33331966, 20473322, 14344292, 10633785, 27417159, 333445, 33550006, 23375157, 2343192, 22230231, 8694863, 937589, 32341147, 15753193, 8299297, 7759108, 14348440, 25083656, 29464864, 27665606, 16953813, 31354877, 22017768, 23665496, 7420324, 11057744, 28755655, 16350026, 6501943, 9529067, 2291438, 31951232, 32453007, 9757, 17136864, 11512989, 2007639, 25951742, 32278189, 32466206, 29017527, 26948055, 18372369, 21794904, 24676330, 32656062, 16260850, 4971230, 27574147, 25997140, 26088784, 31687574, 29311484, 19969308, 5991435, 25370946, 9307116, 29404381, 8470002, 13787848, 31627367, 14312717, 24708569, 9915650, 21855183] 
correct_g = [-1973944623, -176526890, -141899904, -23867736, -35205696, -12302589, -39107780, 9969725, -16106931, 8972626, -12147986, -14406861, 9624562, -11457140, -11346001, -19039429, 509907, 11202994, -8213694, -10963605, -31970506, 6344381, -12400485, 2559018, -3395867, -12935677, 10512582, -4539053, -19458999, -31909767, -2972617, -15568482, -258689, -11862341, 10863695, 16078466, -13709084, 14370032, -6332104, -18818104, -3212633, -1389518, -13911246, -14537635, 2789137, -2105015, -9870725, -7474225, -12948448, 5720141, -4071200, -17476183, 3371612, 6659434, -13677283, -3696750, 11175250, 13431754, 1267007, -3271699, 13903580, -1259566, 9087352, 6266405, -8009315, -4373916, -10801920, -7436977, 998698, 20753169, 9455124, -10064658, -6451758, 8615545, -4133685, -5966341, 17494789, 17827871, -4015080, 19701697, 2656219, -5076595, -8209631, -14302109, 4229918, 13266238, 6693331, -2915122, -15155317, 23451991, -4379716, 8649089, 28928123, -12092893, 8244542, 3041831, -15585620, -9130945, 18981378, 6526926, 12588765, -14708291, -632963, -2287188, 6092608, 170985, 26729523, -11873848, 10743183, 1317436, 11877617, 1337904, -3709966, 7690962, 12589664, 9281077, 2823724, -5664646, 8577431, -2820854, 16307813, 4513950, -16622393, -7044947, -8037363, -4798249, 27437208, -10473010, 30502111, 20527747, 5680604, 17177840, 23621701, 23562652, 25784806, 31827265, 2579508, 23556631, 12186766, 6097479, 26018943, 4631569, 23845164, 6606939, 5906577, 28724629, 16947038, 32010333, 15954371, 7461416, 26888136, 24731234, 19849303, 2344378, 18383022, 32599208, 9469122, 29433582, 13837131, 7988569, 7005506, 29554767, 30191063, 21000992, 32744041, 32956382, 5940822, 1614240, 27992151, 22067442, 27504276, 416063, 11076519, 22220903, 8217000, 4238492, 8322562, 3518523, 1873849, 15121403, 1384976, 4188011, 4485997, 3239919, 5873043, 26052730, 25098895, 24664146, 2119639, 11066474, 2688430, 25014439, 18939177, 23414461, 10810259, 24154559, 11914736, 17311702, 32138356, 7975512, 26561140, 2463690, 9516486, 16947287, 22780288, 18773391, 12378325, 26112142, 17779151, 489130, 15196873, 15522409, 30640414, 17004184, 4916849, 15612116, 28983438, 15874481, 9896306, 28425397, 31174930, 24363167, 30347858, 23028998, 24518416, 1496322, 20601653, 23707147, 14641028, 26666548, 21548237, 12465128, 21950666, 32742777, 20564776, 27648106, 25906930, 4873914, 19191332, 4064170, 2751803, 27265053, 16033356, 5118833, 13848370, 17865431, 13667968, 6445261, 1712468, 22182502, 18403053, 8348736, 1682333, 26608276, 3204539, 24851789] 

check_arrays(f, correct_f,P[0])
check_arrays(g, correct_g,P[0])

not exactly equal
But equal in modulus class
not exactly equal
But equal in modulus class


True